<a href="https://colab.research.google.com/github/kandregulaashok-byte/happy_birthday_kick/blob/main/nasdaq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance pandas requests tqdm


In [ ]:
import yfinance as yf
from datetime import date, timedelta

# --- Dates ---
end_date = date.today() - timedelta(days=1)  # yesterday
start_date = end_date - timedelta(days=120)  # buffer to cover 55 trading days

# --- Fetch historical data for Google ---
df = yf.download("GOOG", start=start_date, end=end_date + timedelta(days=1), interval="1d")

# --- Keep only last 55 trading days ---
last55 = df.tail(55)

# --- Highest high of last 55 trading days ---
highest_55 = last55["High"].max()

# --- Highest high of last 5 trading days ---
highest_5 = last55["High"].tail(5).max()

# --- Results ---
print("Highest high of last 55 trading days (ending yesterday):", highest_55)
print("Highest high of last 5 trading days (ending yesterday):", highest_5)


[*********************100%***********************]  1 of 1 completed

Highest high of last 55 trading days (ending yesterday): Ticker
GOOG    254.139999
dtype: float64
Highest high of last 5 trading days (ending yesterday): Ticker
GOOG    254.139999
dtype: float64


In [ ]:
import yfinance as yf
from datetime import date, timedelta

# --- Dates ---
yesterday = date.today() - timedelta(days=1)
start_date = yesterday - timedelta(days=120)  # buffer for last 55 trading days

# --- Fetch historical data for Google ---
df = yf.download("AMD", start=start_date, end=yesterday + timedelta(days=1), interval="1d")

# --- Last 55 trading days ---
last55 = df.tail(55)
high_series = last55["High"]

# --- Highest highs as scalars ---
highest_55 = high_series.max().item()
highest_5 = high_series.tail(5).max().item()

# --- Check if they are different ---
if highest_55 != highest_5:
    print("✅ Highest highs are different.")

    # Fetch today's data
    today_df = yf.download("AMD", period="1d", interval="1d")
    if not today_df.empty:
        today_close = today_df["Close"].iloc[-1]
        print(f"Today's close: {today_close}")

        # Check if it crossed 55-day high
        if today_close > highest_55:
            print("🔥 Today's closing price crossed the 55-day high!")
        else:
            print("Today's closing price did NOT cross the 55-day high.")
    else:
        print("⚠️ Could not fetch today's data.")
else:
    print("⚠️ Highest highs are equal, no further check.")

# --- Output ---
print("Highest high of last 55 days:", highest_55)
print("Highest high of last 5 days:", highest_5)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

✅ Highest highs are different.
Today's close: Ticker
AMD    156.610001
Name: 2025-09-19 00:00:00, dtype: float64


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
import yfinance as yf
from datetime import date, timedelta

def check_stock(symbol):
    try:
        yesterday = date.today() - timedelta(days=1)
        start_date = yesterday - timedelta(days=120)

        # Fetch historical data
        df = yf.download(symbol, start=start_date, end=yesterday + timedelta(days=1), interval="1d", progress=False)
        if df.empty or "High" not in df.columns:
            print(f"⚠️ No historical data for {symbol}")
            return

        # Ensure High is a Series and drop NaNs
        high_series = df.loc[:, "High"].dropna()
        last55 = high_series.tail(55)
        if last55.empty:
            print(f"⚠️ Not enough valid data for {symbol}")
            return

        # Scalars
        highest_55 = float(last55.max())
        highest_5 = float(last55.tail(5).max())

        print(f"{symbol}: Highest 55-day = {highest_55}, Highest 5-day = {highest_5}")

        if highest_55 != highest_5:
            print("✅ Highest highs are different.")

            # Fetch today's close safely
            today_df = yf.download(symbol, period="1d", interval="1d", progress=False)
            if today_df.empty or "Close" not in today_df.columns:
                print(f"⚠️ Could not fetch today's data for {symbol}")
                return

            today_close = float(today_df["Close"].iloc[-1])
            print(f"Today's close: {today_close}")

            if today_close > highest_55:
                print("🔥 Today's closing price crossed the 55-day high!")
            else:
                print("Today's closing price did NOT cross the 55-day high.")
        else:
            print("⚠️ Highest highs are equal, no further check.")

    except Exception as e:
        print(f"⚠️ Error for {symbol}: {e}")

# --- Example usage ---
for stock in ["GOOG", "TSLA", "AMD", "MSTR"]:
    print("\n--- Checking", stock, "---")
    check_stock(stock)



--- Checking GOOG ---
GOOG: Highest 55-day = 254.13999938964844, Highest 5-day = 254.13999938964844
⚠️ Highest highs are equal, no further check.

--- Checking TSLA ---
TSLA: Highest 55-day = 432.2200012207031, Highest 5-day = 432.2200012207031
⚠️ Highest highs are equal, no further check.

--- Checking AMD ---
AMD: Highest 55-day = 186.64999389648438, Highest 5-day = 162.3000030517578
✅ Highest highs are different.
Today's close: 156.58380126953125
Today's closing price did NOT cross the 55-day high.

--- Checking MSTR ---
MSTR: Highest 55-day = 457.2200012207031, Highest 5-day = 358.25
✅ Highest highs are different.
Today's close: 339.70001220703125
Today's closing price did NOT cross the 55-day high.


In [ ]:
import yfinance as yf
from datetime import date, timedelta

def check_stock(symbol):
    try:
        # --- Dates ---
        end_date = date.today() - timedelta(days=10)  # 10 days back
        start_date = end_date - timedelta(days=120)   # buffer to cover 55 trading days

        # --- Fetch historical data ---
        df = yf.download(symbol, start=start_date, end=end_date + timedelta(days=1), interval="1d", progress=False)
        if df.empty or "High" not in df.columns:
            print(f"⚠️ No historical data for {symbol}")
            return

        # --- High series ---
        high_series = df.loc[:, "High"].dropna()
        last55 = high_series.tail(55)
        if last55.empty:
            print(f"⚠️ Not enough valid data for {symbol}")
            return

        # --- Scalars ---
        highest_55 = float(last55.max())
        highest_5 = float(last55.tail(5).max())

        print(f"{symbol}: Highest 55-day = {highest_55}, Highest 5-day = {highest_5}")

        if highest_55 != highest_5:
            print("✅ Highest highs are different.")

            # Fetch the closing price for the end_date (10 days back)
            close_df = df.loc[end_date:end_date]
            if close_df.empty or "Close" not in close_df.columns:
                print(f"⚠️ Could not fetch closing price for {symbol} on {end_date}")
                return

            closing_price = float(close_df["Close"].iloc[-1])
            print(f"Closing price on {end_date}: {closing_price}")

            if closing_price > highest_55:
                print("🔥 Closing price crossed the 55-day high!")
            else:
                print("Closing price did NOT cross the 55-day high.")
        else:
            print("⚠️ Highest highs are equal, no further check.")

    except Exception as e:
        print(f"⚠️ Error for {symbol}: {e}")

# --- Example usage ---
for stock in ["GOOG", "TSLA", "AMD", "MSTR"]:
    print("\n--- Checking", stock, "---")
    check_stock(stock)



--- Checking GOOG ---
GOOG: Highest 55-day = 240.58999633789062, Highest 5-day = 240.58999633789062
⚠️ Highest highs are equal, no further check.

--- Checking TSLA ---
TSLA: Highest 55-day = 358.44000244140625, Highest 5-day = 358.44000244140625
⚠️ Highest highs are equal, no further check.

--- Checking AMD ---
AMD: Highest 55-day = 186.64999389648438, Highest 5-day = 164.75
✅ Highest highs are different.
Closing price on 2025-09-09: 155.82000732421875
Closing price did NOT cross the 55-day high.

--- Checking MSTR ---
MSTR: Highest 55-day = 457.2200012207031, Highest 5-day = 344.5
✅ Highest highs are different.
Closing price on 2025-09-09: 328.5299987792969
Closing price did NOT cross the 55-day high.


In [ ]:
import yfinance as yf
from datetime import date, timedelta

def check_stock(symbol, days_back=10):
    try:
        # --- Dates ---
        end_date = date.today() - timedelta(days=days_back)  # e.g., 10 days back
        start_date = end_date - timedelta(days=120)          # buffer to cover 55 trading days

        # --- Fetch historical data ---
        df = yf.download(symbol, start=start_date, end=end_date + timedelta(days=1), interval="1d", progress=False)
        if df.empty or "High" not in df.columns:
            print(f"⚠️ No historical data for {symbol}")
            return

        # --- High series ---
        high_series = df.loc[:, "High"].dropna()
        last55 = high_series.tail(55)
        if last55.empty:
            print(f"⚠️ Not enough valid data for {symbol}")
            return

        # --- Scalars ---
        highest_55 = float(last55.max())
        last5 = last55.tail(5)
        highest_5 = float(last5.max())

        # --- Print date ranges for debugging ---
        print(f"\n{symbol} Date Ranges:")
        print(f"55-day range: {last55.index[0].date()} to {last55.index[-1].date()}")
        print(f"5-day range: {last5.index[0].date()} to {last5.index[-1].date()}")

        print(f"\nHighest 55-day high = {highest_55}")
        print(f"Highest 5-day high = {highest_5}")

        if highest_55 != highest_5:
            print("✅ Highest highs are different.")

            # Closing price on the end_date
            close_df = df.loc[end_date:end_date]
            if close_df.empty or "Close" not in close_df.columns:
                print(f"⚠️ Could not fetch closing price for {symbol} on {end_date}")
                return

            closing_price = float(close_df["Close"].iloc[-1])
            print(f"Closing price on {end_date}: {closing_price}")

            if closing_price > highest_55:
                print("🔥 Closing price crossed the 55-day high!")
            else:
                print("Closing price did NOT cross the 55-day high.")
        else:
            print("⚠️ Highest highs are equal, no further check.")

    except Exception as e:
        print(f"⚠️ Error for {symbol}: {e}")

top_performing_stocks = [
    "GRAB"
]

# --- Example usage ---
for stock in top_performing_stocks:
    print("\n--- Checking", stock, "---")
    check_stock(stock, days_back=10)



--- Checking GRAB ---

GRAB Date Ranges:
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09

Highest 55-day high = 5.519999980926514
Highest 5-day high = 5.304999828338623
✅ Highest highs are different.
Closing price on 2025-09-09: 5.289999961853027
Closing price did NOT cross the 55-day high.


In [ ]:
import yfinance as yf
from datetime import date, timedelta

def check_stock(symbol, days_back=10):
    try:
        # --- Dates ---
        end_date = date.today() - timedelta(days=days_back)  # used to calculate 55-day range
        start_date = end_date - timedelta(days=120)          # buffer to cover 55 trading days

        # --- Fetch historical data ---
        df = yf.download(symbol, start=start_date, end=date.today() + timedelta(days=1),
                         interval="1d", progress=False)
        if df.empty or "High" not in df.columns:
            print(f"⚠️ No historical data for {symbol}")
            return

        # --- High series for 55-day and 5-day ranges ending on end_date ---
        high_series = df["High"].dropna()
        last55 = high_series.loc[:end_date].tail(55)
        if last55.empty:
            print(f"⚠️ Not enough valid data for {symbol}")
            return

        highest_55 = float(last55.max())
        last5 = last55.tail(5)
        highest_5 = float(last5.max())

        # --- Print date ranges for debugging ---
        print(f"\n{symbol} Date Ranges:")
        print(f"55-day range: {last55.index[0].date()} to {last55.index[-1].date()}")
        print(f"5-day range: {last5.index[0].date()} to {last5.index[-1].date()}")
        print(f"Highest 55-day high = {highest_55}")
        print(f"Highest 5-day high = {highest_5}")

        if highest_55 != highest_5:
            print("✅ Highest highs are different.")

            # Closing price for today
            today_df = df.loc[date.today():date.today()]
            if today_df.empty or "Close" not in today_df.columns:
                print(f"⚠️ Could not fetch today's closing price for {symbol}")
                return

            closing_price_today = float(today_df["Close"].iloc[-1])
            print(f"Today's closing price: {closing_price_today}")

            if closing_price_today > highest_55:
                print("🔥 Today's closing price crossed the 55-day high!")
            else:
                print("Today's closing price did NOT cross the 55-day high.")
        else:
            print("⚠️ Highest highs are equal, no further check.")

    except Exception as e:
        print(f"⚠️ Error for {symbol}: {e}")

# --- Example usage ---
top_performing_stocks = ["GRAB"]

for stock in top_performing_stocks:
    print("\n--- Checking", stock, "---")
    check_stock(stock, days_back=10)



--- Checking GRAB ---

GRAB Date Ranges:
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09
Highest 55-day high = 5.519999980926514
Highest 5-day high = 5.304999828338623
✅ Highest highs are different.
Today's closing price: 6.389999866485596
🔥 Today's closing price crossed the 55-day high!


In [ ]:
import yfinance as yf
from datetime import date, timedelta

top_performing_stocks = [
    "PLTR", "TPR", "GEV", "VST", "WBD", "AMZN", "SNPS", "MSFT", "FANG", "DHR",
    "BSX", "NWS", "TSLA", "NVDA", "AAPL", "GOOG", "META", "AMD", "PYPL", "NFLX",
    "SPY", "QQQ", "SPCE", "UBER", "LYFT", "BA", "GE", "DIS", "T", "VZ", "CSCO",
    "IBM", "INTC", "AMD", "NVDA", "TSLA", "AAPL", "MSFT", "GOOG", "AMZN", "META",
    "PYPL", "NFLX", "SPY", "QQQ", "SPCE", "UBER", "LYFT", "BA", "GE", "DIS", "T",
    "VZ", "CSCO", "IBM", "INTC", "AMD", "NVDA", "TSLA", "AAPL", "MSFT", "GOOG",
    "AMZN", "META", "PYPL", "NFLX", "SPY", "QQQ", "SPCE", "UBER", "LYFT", "BA",
    "GE", "DIS", "T", "VZ", "CSCO", "IBM", "INTC", "AMD", "NVDA", "TSLA", "AAPL",
    "MSFT", "GOOG", "AMZN", "META", "PYPL", "NFLX", "SPY", "QQQ", "SPCE", "UBER",
    "LYFT", "BA", "GE", "DIS", "T", "VZ", "CSCO", "IBM", "INTC", "AMD", "NVDA",
    "TSLA", "AAPL", "MSFT", "GOOG", "AMZN", "META", "PYPL", "NFLX", "SPY", "QQQ",
    "SPCE", "UBER", "LYFT", "BA", "GE", "DIS", "T", "VZ", "CSCO", "IBM", "INTC",
    "AMD", "NVDA", "TSLA", "AAPL", "MSFT", "GOOG", "AMZN", "META", "PYPL", "NFLX","UPWK","GRAB"
]

def check_stock(symbol, days_back=10):
    try:
        # --- Dates ---
        end_date = date.today() - timedelta(days=days_back)
        start_date = end_date - timedelta(days=120)

        # --- Fetch historical data ---
        df = yf.download(symbol, start=start_date, end=end_date + timedelta(days=1),
                         interval="1d", progress=False)
        if df.empty or "High" not in df.columns:
            return  # safely skip

        # --- High series ---
        high_series = df["High"].dropna()
        last55 = high_series.tail(55)
        if last55.empty:
            return

        highest_55 = float(last55.max())
        last5 = last55.tail(5)
        highest_5 = float(last5.max())

        # --- Only check if 55-day and 5-day highs are different ---
        if highest_55 != highest_5:
            close_df = df.loc[end_date:end_date]
            if close_df.empty or "Close" not in close_df.columns:
                return

            closing_price = float(close_df["Close"].iloc[-1])
            if closing_price > highest_55:
                print(f"🔥 {symbol} CLOSED ABOVE 55-DAY HIGH!")
                print(f"Closing price: {closing_price}, 55-day high: {highest_55}")
                print(f"55-day range: {last55.index[0].date()} to {last55.index[-1].date()}")
                print(f"5-day range: {last5.index[0].date()} to {last5.index[-1].date()}")

    except Exception as e:
        return  # silently skip any errors

# --- Run the check for all stocks ---
for stock in top_performing_stocks:
    check_stock(stock, days_back=10)


In [ ]:
import yfinance as yf
from datetime import date, timedelta

top_performing_stocks = [
    "PLTR", "TPR", "GEV", "VST", "WBD", "AMZN", "SNPS", "MSFT", "FANG", "DHR",
    "BSX", "NWS", "TSLA", "NVDA", "AAPL", "GOOG", "META", "AMD", "PYPL", "NFLX",
    "SPY", "QQQ", "SPCE", "UBER", "LYFT", "BA", "GE", "DIS", "T", "VZ", "CSCO",
    "IBM", "INTC", "UPWK","GRAB"
]

def check_stock(symbol, days_back=10):
    try:
        end_date = date.today() - timedelta(days=days_back)
        start_date = end_date - timedelta(days=120)

        df = yf.download(symbol, start=start_date, end=date.today() + timedelta(days=1),
                         interval="1d", progress=False)
        if df.empty or "High" not in df.columns:
            return

        high_series = df["High"].dropna()
        last55 = high_series.loc[:end_date].tail(55)
        if last55.empty:
            return

        highest_55 = float(last55.max())
        last5 = last55.tail(5)
        highest_5 = float(last5.max())

        if highest_55 != highest_5:
            today_df = df.loc[date.today():date.today()]
            if today_df.empty or "Close" not in today_df.columns:
                return

            closing_price_today = float(today_df["Close"].iloc[-1])

            if closing_price_today > highest_55:
                # --- Print only for stocks breaking 55-day high ---
                print(f"🔥 {symbol} CLOSED ABOVE 55-DAY HIGH!")
                print(f"Today's closing price: {closing_price_today}")
                print(f"55-day high: {highest_55}")
                print(f"5-day high: {highest_5}")
                print(f"55-day range: {last55.index[0].date()} to {last55.index[-1].date()}")
                print(f"5-day range: {last5.index[0].date()} to {last5.index[-1].date()}")
                print("--------------------------------------------------")

    except Exception:
        pass  # silently skip any errors

# --- Run the check for all stocks ---
for stock in top_performing_stocks:
    check_stock(stock, days_back=10)


🔥 TPR CLOSED ABOVE 55-DAY HIGH!
Today's closing price: 114.81999969482422
55-day high: 113.61889161154622
5-day high: 106.0
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09
--------------------------------------------------
🔥 WBD CLOSED ABOVE 55-DAY HIGH!
Today's closing price: 18.948999404907227
55-day high: 13.869999885559082
5-day high: 12.479999542236328
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09
--------------------------------------------------
🔥 QQQ CLOSED ABOVE 55-DAY HIGH!
Today's closing price: 598.6500244140625
55-day high: 583.3200073242188
5-day high: 581.1199951171875
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09
--------------------------------------------------
🔥 UBER CLOSED ABOVE 55-DAY HIGH!
Today's closing price: 97.86000061035156
55-day high: 97.71499633789062
5-day high: 95.4800033569336
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09
----------

In [ ]:
import yfinance as yf
from datetime import date, timedelta

top_performing_stocks = [
    "PLTR", "TPR", "GEV", "VST", "WBD", "AMZN", "SNPS", "MSFT", "FANG", "DHR",
    "BSX", "NWS", "TSLA", "NVDA", "AAPL", "GOOG", "META", "AMD", "PYPL", "NFLX",
    "SPY", "QQQ", "SPCE", "UBER", "LYFT", "BA", "GE", "DIS", "T", "VZ", "CSCO",
    "IBM", "INTC", "UPWK","GRAB"
]

def check_stock(symbol, days_back=10):
    try:
        end_date = date.today() - timedelta(days=days_back)  # 10 days back
        start_date = end_date - timedelta(days=120)          # buffer to cover 55 trading days

        df = yf.download(symbol, start=start_date, end=date.today() + timedelta(days=1),
                         interval="1d", progress=False)
        if df.empty or "High" not in df.columns or "Low" not in df.columns:
            return

        # --- 55-day high ending 10 days ago ---
        high_series = df["High"].dropna()
        last55 = high_series.loc[:end_date].tail(55)
        if last55.empty:
            return
        highest_55 = float(last55.max())

        # --- 5-day high (for info) ---
        last5 = last55.tail(5)
        highest_5 = float(last5.max())

        # --- Low of the last 10 days starting from yesterday ---
        low_series = df["Low"].dropna()
        last10 = low_series.loc[end_date + timedelta(days=1) - timedelta(days=10):end_date]
        if last10.empty:
            return
        lowest_10 = float(last10.min())

        # --- Only check today if lowest_10 < highest_55 ---
        if lowest_10 < highest_55:
            today_df = df.loc[date.today():date.today()]
            if today_df.empty or "Close" not in today_df.columns:
                return

            closing_price_today = float(today_df["Close"].iloc[-1])

            if closing_price_today > highest_55:
                print(f"🔥 {symbol} CLOSED ABOVE 55-DAY HIGH!")
                print(f"Today's closing price: {closing_price_today}")
                print(f"55-day high: {highest_55}")
                print(f"5-day high: {highest_5}")
                print(f"Lowest low of last 10 days: {lowest_10}")
                print(f"55-day range: {last55.index[0].date()} to {last55.index[-1].date()}")
                print(f"5-day range: {last5.index[0].date()} to {last5.index[-1].date()}")
                print(f"10-day range (for low): {last10.index[0].date()} to {last10.index[-1].date()}")
                print("--------------------------------------------------")

    except Exception:
        pass  # silently skip errors

# --- Run the check for all stocks ---
for stock in top_performing_stocks:
    check_stock(stock, days_back=10)


🔥 TPR CLOSED ABOVE 55-DAY HIGH!
Today's closing price: 114.79000091552734
55-day high: 113.61889161154622
5-day high: 106.0
Lowest low of last 10 days: 99.49249063727468
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09
10-day range (for low): 2025-09-02 to 2025-09-09
--------------------------------------------------
🔥 WBD CLOSED ABOVE 55-DAY HIGH!
Today's closing price: 18.934999465942383
55-day high: 13.869999885559082
5-day high: 12.479999542236328
Lowest low of last 10 days: 11.25
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09
10-day range (for low): 2025-09-02 to 2025-09-09
--------------------------------------------------
🔥 TSLA CLOSED ABOVE 55-DAY HIGH!
Today's closing price: 424.71990966796875
55-day high: 358.44000244140625
5-day high: 358.44000244140625
Lowest low of last 10 days: 325.6000061035156
55-day range: 2025-06-23 to 2025-09-09
5-day range: 2025-09-03 to 2025-09-09
10-day range (for low): 2025-09-02 to 2025-0

In [ ]:
import yfinance as yf
from datetime import date, timedelta

symbols = ["AAPL", "GOOG", "TSLA", "AMD", "NVDA", "UPWK", "GRAB"]

for symbol in symbols:
    try:
        # Fetch enough historical data
        start_date = date.today() - timedelta(days=200)
        df = yf.download(symbol, start=start_date, end=date.today() + timedelta(days=1), progress=False)
        if df.empty:
            print(f"⚠️ No data for {symbol}")
            continue

        df = df.sort_index()  # ensure chronological order
        breakout_days = []

        # Loop over last 15 trading days
        for i in range(-15, 0):
            if abs(i) > len(df):
                continue


            current_close = float(df["Close"].iloc[i].iloc[0])  # ensures scalar

            # 55-day high prior to this day
            past_high = df["High"].iloc[:i].tail(55)
            if len(past_high) < 55:
                continue  # skip if not enough data

            highest_55 = float(past_high.max())  # <-- convert to scalar
            highest_55 = float(past_high.max().iloc[0])         # ensures scalar

            if current_close > highest_55:  # scalar comparison now
                breakout_days.append((df.index[i].date(), current_close, highest_55))

        if breakout_days:
            print(f"\n🔥 {symbol} had breakout(s) in last 15 days:")
            for d, close, high55 in breakout_days:
                print(f"{d}: Close={close:.2f} > 55d High={high55:.2f}")
        else:
            print(f"❌ {symbol} no breakout in last 15 days")

    except Exception as e:
        print(f"⚠️ Error for {symbol}: {e}")



🔥 AAPL had breakout(s) in last 15 days:
2025-09-03: Close=238.47 > 55d High=235.12
2025-09-04: Close=239.78 > 55d High=238.85
2025-09-19: Close=245.50 > 55d High=241.32

🔥 GOOG had breakout(s) in last 15 days:
2025-08-29: Close=213.34 > 55d High=212.71
2025-09-03: Close=230.89 > 55d High=215.15
2025-09-04: Close=232.45 > 55d High=231.59
2025-09-05: Close=234.96 > 55d High=232.56
2025-09-09: Close=239.94 > 55d High=238.40
2025-09-15: Close=251.76 > 55d High=242.57
2025-09-19: Close=255.24 > 55d High=254.14

🔥 TSLA had breakout(s) in last 15 days:
2025-09-11: Close=368.81 > 55d High=358.44
2025-09-12: Close=395.94 > 55d High=368.99
2025-09-15: Close=410.04 > 55d High=396.69
2025-09-17: Close=425.86 > 55d High=425.70
❌ AMD no breakout in last 15 days
❌ NVDA no breakout in last 15 days

🔥 UPWK had breakout(s) in last 15 days:
2025-09-02: Close=15.96 > 55d High=15.41
2025-09-03: Close=16.45 > 55d High=16.00
2025-09-04: Close=16.78 > 55d High=16.57
2025-09-17: Close=18.14 > 55d High=17.29
2

In [ ]:
import yfinance as yf
from datetime import date, timedelta
import pandas as pd

top_performing_stocks = [
    "AAPL", "GOOG", "TSLA", "AMD", "NVDA", "UPWK", "GRAB"
]

# --- Function to check 55-day breakout with gap ---
def check_breakout_with_gap(symbol, lookback_days=15, window=55):
    try:
        # --- Download data ---
        start_date = date.today() - timedelta(days=200)  # buffer for 55-day lookback
        df = yf.download(symbol, start=start_date, end=date.today() + timedelta(days=1), progress=False)

        # Flatten columns if MultiIndex
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        # Check required columns
        required_cols = ['High', 'Low', 'Close']
        for col in required_cols:
            if col not in df.columns:
                print(f"⚠️ Error for {symbol}: missing column {col}")
                return

        print(f"✅ {symbol} data ready, {len(df)} rows")

        breakouts = []
        recent_df = df.tail(lookback_days + window)  # extra buffer for 55-day lookback

        for i in range(window, len(recent_df)-1):  # avoid last row for next-day gap check
            current_close = float(recent_df["Close"].iloc[i])
            past_high = recent_df["High"].iloc[i-window:i]
            highest_55 = float(past_high.max())

            if current_close > highest_55:
                # Gap check: previous day high vs next day low
                prev_high = float(recent_df["High"].iloc[i-1])
                next_low = float(recent_df["Low"].iloc[i+1])
                gap_exists = next_low > prev_high

                breakout_date = recent_df.index[i].date()
                breakouts.append({
                    "date": breakout_date,
                    "close": current_close,
                    "55d_high": highest_55,
                    "gap_next_day": gap_exists
                })

        if breakouts:
            print(f"\n🔥 {symbol} had breakout(s) in last {lookback_days} days:")
            for b in breakouts:
                gap_status = "✅ Gap exists" if b["gap_next_day"] else "❌ No gap"
                print(f"{b['date']}: Close={b['close']} > 55d High={b['55d_high']} ({gap_status})")
        else:
            print(f"❌ {symbol} no breakout in last {lookback_days} days")

    except Exception as e:
        print(f"⚠️ Error for {symbol}: {e}")


# --- Run for all stocks ---
for stock in top_performing_stocks:
    check_breakout_with_gap(stock)


✅ AAPL data ready, 139 rows

🔥 AAPL had breakout(s) in last 15 days:
2025-09-03: Close=238.47000122070312 > 55d High=235.1199951171875 (✅ Gap exists)
2025-09-04: Close=239.77999877929688 > 55d High=238.85000610351562 (❌ No gap)
✅ GOOG data ready, 139 rows

🔥 GOOG had breakout(s) in last 15 days:
2025-08-29: Close=213.3393096923828 > 55d High=212.70986910562976 (❌ No gap)
2025-09-03: Close=230.8936309814453 > 55d High=215.14769086553312 (✅ Gap exists)
2025-09-04: Close=232.45223999023438 > 55d High=231.59300282388122 (✅ Gap exists)
2025-09-05: Close=234.95999145507812 > 55d High=232.56214237085004 (✅ Gap exists)
2025-09-09: Close=239.94000244140625 > 55d High=238.39999389648438 (❌ No gap)
2025-09-15: Close=251.75999450683594 > 55d High=242.57000732421875 (✅ Gap exists)
✅ TSLA data ready, 139 rows

🔥 TSLA had breakout(s) in last 15 days:
2025-09-11: Close=368.80999755859375 > 55d High=358.44000244140625 (✅ Gap exists)
2025-09-12: Close=395.94000244140625 > 55d High=368.989990234375 (✅ Ga

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days)  # need 1 yr + 30 days data

# Example NASDAQ tickers (replace with full list if available)
tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

results = []

def check_fvg_at_breakout(df, ticker):
    df['52w_high'] = df['High'].rolling(window=252, min_periods=1).max()

    for i in range(1, len(df)-1):  # need A(i-1), B(i), C(i+1)
        date = df.index[i]
        if (today - date.date()).days > lookback_days:
            continue

        # Get scalar values instead of Series
        close_B = float(df['Close'].iloc[i])
        prev_52w_high = float(df['52w_high'].iloc[i-1])

        # breakout condition: B closes above prev 52w high
        if close_B > prev_52w_high:
            A = df.iloc[i-1]  # candle before breakout
            B = df.iloc[i]    # breakout candle
            C = df.iloc[i+1]  # candle after breakout

            # FVG check (scalar values)
            if float(C['Low']) > float(A['High']):
                results.append({
                    "Ticker": ticker,
                    "Breakout_Date(B)": date.date(),
                    "52w_High_Prev": prev_52w_high,
                    "Breakout_Close(B)": close_B,
                    "Candle_A_Date": df.index[i-1].date(),
                    "Candle_A_High": float(A['High']),
                    "Candle_C_Date": df.index[i+1].date(),
                    "Candle_C_Low": float(C['Low']),
                    "FVG_Zone": f"{float(A['High'])} - {float(C['Low'])}"
                })

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# Save results
df_results = pd.DataFrame(results)
print(df_results)
df_results.to_csv("nasdaq_fvg_breakouts.csv", index=False)


/tmp/ipython-input-1697160540.py:50: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=today)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1697160540.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  close_B = float(df['Close'].iloc[i])
/tmp/ipython-input-1697160540.py:36: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if float(C['Low']) > float(A['High']):
/tmp/ipython-input-1697160540.py:43: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "Candle_A_High": float(A['High']),
/tmp/ipython-input-1697160540.py:45: FutureWarning: Calling float on a single element Series is depre

  Ticker Breakout_Date(B)  52w_High_Prev  Breakout_Close(B) Candle_A_Date  \
0   SOFI       2025-08-25      25.250000          26.150000    2025-08-22   
1   SOFI       2025-09-18      27.879999          28.110001    2025-09-17   

   Candle_A_High Candle_C_Date  Candle_C_Low  \
0      25.250000    2025-08-26         25.50   
1      27.459999    2025-09-19         28.24   

                                  FVG_Zone  
0                             25.25 - 25.5  
1  27.459999084472656 - 28.239999771118164  



/tmp/ipython-input-1697160540.py:26: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  close_B = float(df['Close'].iloc[i])


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days)  # need 1 yr + 30 days data

# Example NASDAQ tickers (replace with full list if available)
tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

results = []

def check_fvg_at_breakout(df, ticker):
    df['52w_high'] = df['High'].rolling(window=252, min_periods=1).max()

    for i in range(1, len(df)-1):  # need A(i-1), B(i), C(i+1)
        date = df.index[i]
        if (today - date.date()).days > lookback_days:
            continue

        # Get scalar values using .item()
        close_B = df['Close'].iloc[i].item()
        prev_52w_high = df['52w_high'].iloc[i-1].item()

        # breakout condition: B closes above prev 52w high
        if close_B > prev_52w_high:
            A = df.iloc[i-1]  # candle before breakout
            B = df.iloc[i]    # breakout candle
            C = df.iloc[i+1]  # candle after breakout

            A_high = A['High'].item()
            C_low = C['Low'].item()

            # Bullish FVG check
            if C_low > A_high:
                results.append({
                    "Ticker": ticker,
                    "Breakout_Date(B)": date.date(),
                    "52w_High_Prev": prev_52w_high,
                    "Breakout_Close(B)": close_B,
                    "Candle_A_Date": df.index[i-1].date(),
                    "Candle_A_High": A_high,
                    "Candle_C_Date": df.index[i+1].date(),
                    "Candle_C_Low": C_low,
                    "FVG_Zone": f"{A_high} - {C_low}"
                })

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=True)  # explicit auto_adjust
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# Save results
df_results = pd.DataFrame(results)
print(df_results)
df_results.to_csv("nasdaq_fvg_breakouts.csv", index=False)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

  Ticker Breakout_Date(B)  52w_High_Prev  Breakout_Close(B) Candle_A_Date  \
0   SOFI       2025-08-25      25.250000          26.150000    2025-08-22   
1   SOFI       2025-09-18      27.879999          28.110001    2025-09-17   

   Candle_A_High Candle_C_Date  Candle_C_Low  \
0      25.250000    2025-08-26         25.50   
1      27.459999    2025-09-19         28.24   

                                  FVG_Zone  
0                             25.25 - 25.5  
1  27.459999084472656 - 28.239999771118164  


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days)  # need 1 yr + 30 days data

# Example NASDAQ tickers (replace with full list if available)
tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

# Store results
df_results = []

# ---------------------------
# Function to check FVG at breakout
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df['52w_high'] = df['High'].rolling(window=252, min_periods=1).max()

    for i in range(1, len(df)-1):  # need A(i-1), B(i), C(i+1)
        date = df.index[i]
        if (today - date.date()).days > lookback_days:
            continue

        # Get scalar values using .item()
        close_B = df['Close'].iloc[i].item()
        prev_52w_high = df['52w_high'].iloc[i-1].item()

        # Breakout condition: B closes above prev 52w high
        if close_B > prev_52w_high:
            A = df.iloc[i-1]  # candle before breakout
            B = df.iloc[i]    # breakout candle
            C = df.iloc[i+1]  # candle after breakout

            A_high = A['High'].item()
            C_low = C['Low'].item()

            # Bullish FVG check
            if C_low > A_high:
                df_results.append({
                    "Ticker": ticker,
                    "Breakout_Date(B)": date.date(),
                    "52w_High_Prev": prev_52w_high,
                    "Breakout_Close(B)": close_B,
                    "Candle_A_Date": df.index[i-1].date(),
                    "Candle_A_High": A_high,
                    "Candle_C_Date": df.index[i+1].date(),
                    "Candle_C_Low": C_low,
                    "FVG_Zone": f"{A_high} → {C_low}"
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=True)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Format and Display Results
# ---------------------------
if df_results:
    df_results = pd.DataFrame(df_results)

    # Round numeric columns
    df_results["52w_High_Prev"] = df_results["52w_High_Prev"].round(2)
    df_results["Breakout_Close(B)"] = df_results["Breakout_Close(B)"].round(2)
    df_results["Candle_A_High"] = df_results["Candle_A_High"].round(2)
    df_results["Candle_C_Low"] = df_results["Candle_C_Low"].round(2)

    # Rename & reorder columns
    df_results = df_results[
        [
            "Ticker",
            "Breakout_Date(B)",
            "52w_High_Prev",
            "Breakout_Close(B)",
            "Candle_A_Date",
            "Candle_A_High",
            "Candle_C_Date",
            "Candle_C_Low",
            "FVG_Zone",
        ]
    ].rename(
        columns={
            "Ticker": "Stock",
            "Breakout_Date(B)": "Breakout Date",
            "52w_High_Prev": "Prev 52W High",
            "Breakout_Close(B)": "Breakout Close",
            "Candle_A_Date": "Candle A (Date)",
            "Candle_A_High": "Candle A High",
            "Candle_C_Date": "Candle C (Date)",
            "Candle_C_Low": "Candle C Low",
            "FVG_Zone": "FVG Zone",
        }
    )

    print("\n📊 Breakout + FVG Results:\n")
    print(df_results.to_string(index=False))

    # Save to CSV
    df_results.to_csv("nasdaq_fvg_breakouts.csv", index=False)
else:
    print("⚠️ No breakouts with FVG found in the last 30 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



📊 Breakout + FVG Results:

Stock Breakout Date  Prev 52W High  Breakout Close Candle A (Date)  Candle A High Candle C (Date)  Candle C Low                                FVG Zone
 SOFI    2025-08-25          25.25           26.15      2025-08-22          25.25      2025-08-26         25.50                            25.25 → 25.5
 SOFI    2025-09-18          27.88           28.11      2025-09-17          27.46      2025-09-19         28.24 27.459999084472656 → 28.239999771118164


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30  # look for breakouts in last 30 days
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days + 30)  # extra buffer for pre-30d 52w high

# Example NASDAQ tickers (replace with full list if available)
tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

# Store results
df_results = []

# ---------------------------
# Function to check FVG at breakout
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):  # need A(i-1), B(i), C(i+1)
        date = df.index[i]
        if (today - date.date()).days > lookback_days:
            continue

        B = df.iloc[i]       # breakout candle
        A = df.iloc[i-1]     # candle before breakout
        C = df.iloc[i+1]     # candle after breakout

        # Extract scalar values safely
        close_B = B['Close'].item()
        A_high = A['High'].item()
        C_low = C['Low'].item()

        # Define 52-week window ending 30 days before breakout
        pre_30_index = max(i - 30, 0)
        start_index = max(pre_30_index - 252, 0)  # 252 trading days ≈ 1 year
        prev_52w_high_slice = df['High'].iloc[start_index:pre_30_index]
        if prev_52w_high_slice.empty:
            continue
        prev_52w_high = prev_52w_high_slice.max()
        if isinstance(prev_52w_high, pd.Series):
            prev_52w_high = prev_52w_high.item()

        # Breakout condition: B closes above pre-30-days 52w high
        if close_B > prev_52w_high:
            # Bullish FVG check
            if C_low > A_high:
                df_results.append({
                    "Ticker": ticker,
                    "Breakout_Date(B)": date.date(),
                    "Prev_52W_High_Pre30": round(prev_52w_high, 2),
                    "Breakout_Close(B)": round(close_B, 2),
                    "Candle_A_Date": A.name.date(),
                    "Candle_A_High": round(A_high, 2),
                    "Candle_C_Date": C.name.date(),
                    "Candle_C_Low": round(C_low, 2),
                    "FVG_Zone": f"{round(A_high,2)} → {round(C_low,2)}"
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=True)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Format and Display Results
# ---------------------------
if df_results:
    df_results = pd.DataFrame(df_results)

    # Reorder & rename columns
    df_results = df_results[
        [
            "Ticker",
            "Breakout_Date(B)",
            "Prev_52W_High_Pre30",
            "Breakout_Close(B)",
            "Candle_A_Date",
            "Candle_A_High",
            "Candle_C_Date",
            "Candle_C_Low",
            "FVG_Zone",
        ]
    ].rename(
        columns={
            "Ticker": "Stock",
            "Breakout_Date(B)": "Breakout Date",
            "Prev_52W_High_Pre30": "Prev 52W High (Before 30d)",
            "Breakout_Close(B)": "Breakout Close",
            "Candle_A_Date": "Candle A (Date)",
            "Candle_A_High": "Candle A High",
            "Candle_C_Date": "Candle C (Date)",
            "Candle_C_Low": "Candle C Low",
            "FVG_Zone": "FVG Zone",
        }
    )

    print("\n📊 52-Week Breakout + FVG Results:\n")
    print(df_results.to_string(index=False))

    # Save to CSV
    df_results.to_csv("nasdaq_fvg_breakouts.csv", index=False)
else:
    print("⚠️ No breakouts with FVG found in the last 30 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


📊 52-Week Breakout + FVG Results:

Stock Breakout Date  Prev 52W High (Before 30d)  Breakout Close Candle A (Date)  Candle A High Candle C (Date)  Candle C Low      FVG Zone
 SOFI    2025-08-21                       20.89           23.33      2025-08-20          22.72      2025-08-22         22.90  22.72 → 22.9
 SOFI    2025-08-22                       21.32           25.02      2025-08-21          23.37      2025-08-25         24.81 23.37 → 24.81
 SOFI    2025-08-25                       21.86           26.15      2025-08-22          25.25      2025-08-26         25.50  25.25 → 25.5
 SOFI    2025-09-12                       25.11           26.55      2025-09-11          26.60      2025-09-15         26.93  26.6 → 26.93
 SOFI    2025-09-18                       25.11           28.11      2025-09-17          27.46      2025-09-19         28.24 27.46 → 28.24


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30  # look for breakouts in last 30 days
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days + 30)  # extra buffer for pre-30d 52w high

# Example NASDAQ tickers (replace with full list if available)
tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

# Store results
df_results = []

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):  # need A(i-1), B(i), C(i+1)
        B = df.iloc[i]
        A = df.iloc[i-1]
        C = df.iloc[i+1]

        date = df.index[i].date()

        # Extract scalars
        close_B = B['Close'].item()
        A_high = A['High'].item()
        C_low = C['Low'].item()

        # 52-week high window ending 30 days before breakout
        pre_30_index = max(i - 30, 0)
        start_index = max(pre_30_index - 252, 0)  # 252 trading days ≈ 1 year
        prev_52w_high_slice = df['High'].iloc[start_index:pre_30_index]
        if prev_52w_high_slice.empty:
            continue
        prev_52w_high = prev_52w_high_slice.max()
        if isinstance(prev_52w_high, pd.Series):
            prev_52w_high = prev_52w_high.item()

        # Breakout condition
        if close_B > prev_52w_high:
            # Bullish FVG check
            if C_low > A_high:
                fvg_low = A_high
                fvg_high = C_low
                fvg_date = date

                # Look forward for FVG test
                test_date = None
                test_low = None
                for j in range(i+1, total_len):
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break  # first test only

                df_results.append({
                    "Ticker": ticker,
                    "Breakout_Date(B)": fvg_date,
                    "Prev_52W_High_Pre30": round(prev_52w_high, 2),
                    "Breakout_Close(B)": round(close_B, 2),
                    "Candle_A_Date": A.name.date(),
                    "Candle_A_High": round(A_high, 2),
                    "Candle_C_Date": C.name.date(),
                    "Candle_C_Low": round(C_low, 2),
                    "FVG_Zone": f"{round(A_high,2)} → {round(C_low,2)}",
                    "FVG_Test_Date": test_date,
                    "FVG_Test_Low": round(test_low,2) if test_low else None
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=True)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Format and Display Results
# ---------------------------
if df_results:
    df_results = pd.DataFrame(df_results)

    # Reorder & rename columns
    df_results = df_results[
        [
            "Ticker",
            "Breakout_Date(B)",
            "Prev_52W_High_Pre30",
            "Breakout_Close(B)",
            "Candle_A_Date",
            "Candle_A_High",
            "Candle_C_Date",
            "Candle_C_Low",
            "FVG_Zone",
            "FVG_Test_Date",
            "FVG_Test_Low"
        ]
    ].rename(
        columns={
            "Ticker": "Stock",
            "Breakout_Date(B)": "Breakout Date",
            "Prev_52W_High_Pre30": "Prev 52W High (Before 30d)",
            "Breakout_Close(B)": "Breakout Close",
            "Candle_A_Date": "Candle A (Date)",
            "Candle_A_High": "Candle A High",
            "Candle_C_Date": "Candle C (Date)",
            "Candle_C_Low": "Candle C Low",
            "FVG_Zone": "FVG Zone",
            "FVG_Test_Date": "FVG First Test Date",
            "FVG_Test_Low": "FVG First Test Low"
        }
    )

    print("\n📊 52-Week Breakout + FVG + FVG Test Results:\n")
    print(df_results.to_string(index=False))

    # Save to CSV
    df_results.to_csv("nasdaq_fvg_breakouts_with_tests.csv", index=False)
else:
    print("⚠️ No breakouts with FVG found in the last 30 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



📊 52-Week Breakout + FVG + FVG Test Results:

Stock Breakout Date  Prev 52W High (Before 30d)  Breakout Close Candle A (Date)  Candle A High Candle C (Date)  Candle C Low        FVG Zone FVG First Test Date  FVG First Test Low
 SOFI    2024-09-16                        7.95            8.06      2024-09-13           7.74      2024-09-17          8.06     7.74 → 8.06          2024-09-17                8.06
 SOFI    2024-10-04                        7.95            8.39      2024-10-03           7.86      2024-10-07          8.16     7.86 → 8.16          2024-10-07                8.16
 SOFI    2024-10-11                        7.95            9.01      2024-10-10           8.76      2024-10-14          9.38     8.76 → 9.38          2024-10-14                9.38
 SOFI    2024-10-14                        8.30           10.04      2024-10-11           9.07      2024-10-15         10.03    9.07 → 10.03          2024-10-15               10.03
 SOFI    2024-10-18                        8.30 

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30  # look for breakouts in last 30 days
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days + 30)  # extra buffer for pre-30d 52w high

# Example NASDAQ tickers (replace with full list if available)
tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

# Store results
df_results = []

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):  # need A(i-1), B(i), C(i+1)
        B = df.iloc[i]
        A = df.iloc[i-1]
        C = df.iloc[i+1]

        date = df.index[i].date()

        # Extract scalars
        B_close = B['Close'].item()
        B_open = B['Open'].item()
        A_high = A['High'].item()
        C_low = C['Low'].item()

        # 52-week high window ending 30 days before breakout
        pre_30_index = max(i - 30, 0)
        start_index = max(pre_30_index - 252, 0)  # 252 trading days ≈ 1 year
        prev_52w_high_slice = df['High'].iloc[start_index:pre_30_index]
        if prev_52w_high_slice.empty:
            continue
        prev_52w_high = prev_52w_high_slice.max()
        if isinstance(prev_52w_high, pd.Series):
            prev_52w_high = prev_52w_high.item()

        # Breakout condition: must break 52-week high and be bullish
        if B_close > prev_52w_high and B_close > B_open:
            # Bullish FVG check
            if C_low > A_high:
                fvg_low = A_high
                fvg_high = C_low
                fvg_date = date

                # Look forward for FVG test
                test_date = None
                test_low = None
                for j in range(i+1, total_len):
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break  # first test only

                df_results.append({
                    "Ticker": ticker,
                    "Breakout_Date(B)": fvg_date,
                    "Prev_52W_High_Pre30": round(prev_52w_high, 2),
                    "Breakout_Close(B)": round(B_close, 2),
                    "Candle_A_Date": A.name.date(),
                    "Candle_A_High": round(A_high, 2),
                    "Candle_C_Date": C.name.date(),
                    "Candle_C_Low": round(C_low, 2),
                    "FVG_Zone": f"{round(A_high,2)} → {round(C_low,2)}",
                    "FVG_Test_Date": test_date,
                    "FVG_Test_Low": round(test_low,2) if test_low else None
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=True)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Format and Display Results
# ---------------------------
if df_results:
    df_results = pd.DataFrame(df_results)

    # Reorder & rename columns
    df_results = df_results[
        [
            "Ticker",
            "Breakout_Date(B)",
            "Prev_52W_High_Pre30",
            "Breakout_Close(B)",
            "Candle_A_Date",
            "Candle_A_High",
            "Candle_C_Date",
            "Candle_C_Low",
            "FVG_Zone",
            "FVG_Test_Date",
            "FVG_Test_Low"
        ]
    ].rename(
        columns={
            "Ticker": "Stock",
            "Breakout_Date(B)": "Breakout Date",
            "Prev_52W_High_Pre30": "Prev 52W High (Before 30d)",
            "Breakout_Close(B)": "Breakout Close",
            "Candle_A_Date": "Candle A (Date)",
            "Candle_A_High": "Candle A High",
            "Candle_C_Date": "Candle C (Date)",
            "Candle_C_Low": "Candle C Low",
            "FVG_Zone": "FVG Zone",
            "FVG_Test_Date": "FVG First Test Date",
            "FVG_Test_Low": "FVG First Test Low"
        }
    )

    print("\n📊 52-Week Bullish Breakout + FVG + FVG Test Results:\n")
    print(df_results.to_string(index=False))

    # Save to CSV
    df_results.to_csv("nasdaq_fvg_bullish_breakouts_with_tests.csv", index=False)
else:
    print("⚠️ No bullish breakouts with FVG found in the last 30 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



📊 52-Week Bullish Breakout + FVG + FVG Test Results:

Stock Breakout Date  Prev 52W High (Before 30d)  Breakout Close Candle A (Date)  Candle A High Candle C (Date)  Candle C Low        FVG Zone FVG First Test Date  FVG First Test Low
 SOFI    2024-09-16                        7.95            8.06      2024-09-13           7.74      2024-09-17          8.06     7.74 → 8.06          2024-09-17                8.06
 SOFI    2024-10-04                        7.95            8.39      2024-10-03           7.86      2024-10-07          8.16     7.86 → 8.16          2024-10-07                8.16
 SOFI    2024-10-11                        7.95            9.01      2024-10-10           8.76      2024-10-14          9.38     8.76 → 9.38          2024-10-14                9.38
 SOFI    2024-10-14                        8.30           10.04      2024-10-11           9.07      2024-10-15         10.03    9.07 → 10.03          2024-10-15               10.03
 SOFI    2024-10-18                     

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30  # look for breakouts in last 30 days
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days + 30)  # buffer for pre-30d 52w high

# Example NASDAQ tickers (replace with full list if available)
tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

# Store results
df_results = []

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):  # need A(i-1), B(i), C(i+1)
        B = df.iloc[i]
        A = df.iloc[i-1]
        C = df.iloc[i+1]

        date = df.index[i].date()

        # Extract scalars
        B_close = B['Close'].item()
        B_open = B['Open'].item()
        A_high = A['High'].item()
        C_low = C['Low'].item()

        # 52-week high window ending 30 days before breakout
        pre_30_index = max(i - 30, 0)
        start_index = max(pre_30_index - 252, 0)  # 252 trading days ≈ 1 year
        prev_52w_high_slice = df['High'].iloc[start_index:pre_30_index]
        if prev_52w_high_slice.empty:
            continue
        prev_52w_high = prev_52w_high_slice.max()
        if isinstance(prev_52w_high, pd.Series):
            prev_52w_high = prev_52w_high.item()

        # Recent 30-day high
        recent_30d_slice = df['High'].iloc[pre_30_index:i]
        recent_30d_high = recent_30d_slice.max()
        if isinstance(recent_30d_high, pd.Series):
            recent_30d_high = recent_30d_high.item()

        # Breakout condition:
        # 1) closes above pre-30-day 52-week high
        # 2) bullish candle
        # 3) recent 30-day high < 52-week high
        if B_close > prev_52w_high and B_close > B_open and recent_30d_high < prev_52w_high:
            # Bullish FVG check
            if C_low > A_high:
                fvg_low = A_high
                fvg_high = C_low
                fvg_date = date

                # Look forward for FVG test
                test_date = None
                test_low = None
                for j in range(i+1, total_len):
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break  # first test only

                df_results.append({
                    "Ticker": ticker,
                    "Breakout_Date(B)": fvg_date,
                    "Prev_52W_High_Pre30": round(prev_52w_high, 2),
                    "Breakout_Close(B)": round(B_close, 2),
                    "Recent_30D_High": round(recent_30d_high, 2),
                    "Candle_A_Date": A.name.date(),
                    "Candle_A_High": round(A_high, 2),
                    "Candle_C_Date": C.name.date(),
                    "Candle_C_Low": round(C_low, 2),
                    "FVG_Zone": f"{round(A_high,2)} → {round(C_low,2)}",
                    "FVG_Test_Date": test_date,
                    "FVG_Test_Low": round(test_low,2) if test_low else None
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=True)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Format and Display Results
# ---------------------------
if df_results:
    df_results = pd.DataFrame(df_results)

    # Reorder & rename columns
    df_results = df_results[
        [
            "Ticker",
            "Breakout_Date(B)",
            "Prev_52W_High_Pre30",
            "Recent_30D_High",
            "Breakout_Close(B)",
            "Candle_A_Date",
            "Candle_A_High",
            "Candle_C_Date",
            "Candle_C_Low",
            "FVG_Zone",
            "FVG_Test_Date",
            "FVG_Test_Low"
        ]
    ].rename(
        columns={
            "Ticker": "Stock",
            "Breakout_Date(B)": "Breakout Date",
            "Prev_52W_High_Pre30": "Prev 52W High (Before 30d)",
            "Recent_30D_High": "Recent 30-Day High",
            "Breakout_Close(B)": "Breakout Close",
            "Candle_A_Date": "Candle A (Date)",
            "Candle_A_High": "Candle A High",
            "Candle_C_Date": "Candle C (Date)",
            "Candle_C_Low": "Candle C Low",
            "FVG_Zone": "FVG Zone",
            "FVG_Test_Date": "FVG First Test Date",
            "FVG_Test_Low": "FVG First Test Low"
        }
    )

    print("\n📊 52-Week Bullish Breakout + FVG + FVG Test Results:\n")
    print(df_results.to_string(index=False))

    # Save to CSV
    df_results.to_csv("nasdaq_fvg_bullish_breakouts_with_tests.csv", index=False)
else:
    print("⚠️ No valid bullish breakouts with FVG found in the last 30 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



📊 52-Week Bullish Breakout + FVG + FVG Test Results:

Stock Breakout Date  Prev 52W High (Before 30d)  Recent 30-Day High  Breakout Close Candle A (Date)  Candle A High Candle C (Date)  Candle C Low        FVG Zone FVG First Test Date  FVG First Test Low
 AAPL    2024-11-29                      236.39              235.75          236.49      2024-11-27         234.86      2024-12-02        236.32 234.86 → 236.32          2024-12-02              236.32


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30  # last 30 days
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days + 30)  # buffer for prev 52w high

tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

df_results = []

# Minimum thresholds
min_breakout_pct = 0.005  # 0.5% above 52-week high
min_fvg_pct = 0.005       # FVG at least 0.5% above A_high

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):  # need A(i-1), B(i), C(i+1)
        B = df.iloc[i]
        A = df.iloc[i-1]
        C = df.iloc[i+1]

        B_close = B['Close'].item()
        B_open = B['Open'].item()
        A_high = A['High'].item()
        C_low = C['Low'].item()
        date = df.index[i].date()

        # 52-week high window ending 30 days before breakout
        pre_30_index = max(i - 30, 0)
        start_index = max(pre_30_index - 252, 0)
        prev_52w_high_slice = df['High'].iloc[start_index:pre_30_index]
        if prev_52w_high_slice.empty:
            continue
        prev_52w_high = prev_52w_high_slice.max()
        if isinstance(prev_52w_high, pd.Series):
            prev_52w_high = prev_52w_high.item()

        # Recent 30-day high
        recent_30d_slice = df['High'].iloc[pre_30_index:i]
        recent_30d_high = recent_30d_slice.max()
        if isinstance(recent_30d_high, pd.Series):
            recent_30d_high = recent_30d_high.item()

        # ---------------------------
        # Breakout Conditions
        # ---------------------------
        if (
            B_close > prev_52w_high * (1 + min_breakout_pct) and  # min breakout threshold
            B_close > B_open and                                 # bullish candle
            recent_30d_high < prev_52w_high                     # recent highs below 52-week
        ):
            # ---------------------------
            # FVG Conditions
            # ---------------------------
            if C_low > A_high * (1 + min_fvg_pct):
                fvg_low = A_high
                fvg_high = C_low
                fvg_date = date

                # Look forward for FVG test
                test_date = None
                test_low = None
                for j in range(i+1, total_len):
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break  # first test only

                df_results.append({
                    "Stock": ticker,
                    "Breakout Date": fvg_date,
                    "Prev 52W High (Before 30d)": prev_52w_high,
                    "Recent 30-Day High": recent_30d_high,
                    "Breakout Close": B_close,
                    "Candle A (Date)": A.name.date(),
                    "Candle A High": A_high,
                    "Candle C (Date)": C.name.date(),
                    "Candle C Low": C_low,
                    "FVG Zone": f"{A_high} → {C_low}",
                    "FVG First Test Date": test_date,
                    "FVG First Test Low": test_low
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=False)  # raw prices
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Transposed Output
# ---------------------------
if df_results:
    for res in df_results:
        print("\n============================")
        print(f"📊 52-Week Bullish Breakout + FVG + FVG Test: {res['Stock']}")
        print("----------------------------")
        for k, v in res.items():
            if k != "Stock":
                print(f"{k}: {v}")
    # Optional: save to CSV
    pd.DataFrame(df_results).to_csv("nasdaq_fvg_bullish_breakouts_transposed.csv", index=False)
else:
    print("⚠️ No valid bullish breakouts with meaningful FVG found in the last 30 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


⚠️ No valid bullish breakouts with meaningful FVG found in the last 30 days.


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30  # last 30 days
today = datetime.today().date()
start_date = today - timedelta(days=55 + lookback_days + 30)  # buffer for prev 55d high

tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT","GOOG"]

df_results = []

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):
        B = df.iloc[i]
        A = df.iloc[i-1]
        C = df.iloc[i+1]

        B_close = B['Close'].item()
        B_open = B['Open'].item()
        A_high = A['High'].item()
        C_low = C['Low'].item()
        date = df.index[i].date()

        # 55-day high window before breakout
        prev_55d_slice = df['High'].iloc[max(i-55,0):i]
        if prev_55d_slice.empty:
            continue
        prev_55d_high = prev_55d_slice.max()
        if isinstance(prev_55d_high, pd.Series):
            prev_55d_high = prev_55d_high.item()

        # Recent 30-day high
        recent_30d_slice = df['High'].iloc[max(i-30,0):i]
        recent_30d_high = recent_30d_slice.max()
        if isinstance(recent_30d_high, pd.Series):
            recent_30d_high = recent_30d_high.item()

        # ---------------------------
        # Breakout Conditions
        # ---------------------------
        if (
            B_close > prev_55d_high and   # simple breakout above 55-day high
            B_close > B_open and          # bullish candle
            recent_30d_high < prev_55d_high
        ):
            # ---------------------------
            # FVG Conditions
            # ---------------------------
            if C_low > A_high:  # no minimum % threshold
                fvg_low = A_high
                fvg_high = C_low
                fvg_date = date

                # Look forward for FVG test
                test_date = None
                test_low = None
                for j in range(i+1, total_len):
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break

                df_results.append({
                    "Stock": ticker,
                    "Breakout Date": fvg_date,
                    "Prev 55D High": prev_55d_high,
                    "Recent 30-Day High": recent_30d_high,
                    "Breakout Close": B_close,
                    "Candle A (Date)": A.name.date(),
                    "Candle A High": A_high,
                    "Candle C (Date)": C.name.date(),
                    "Candle C Low": C_low,
                    "FVG Zone": f"{A_high} → {C_low}",
                    "FVG First Test Date": test_date,
                    "FVG First Test Low": test_low
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=False)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Transposed Output
# ---------------------------
if df_results:
    for res in df_results:
        print("\n============================")
        print(f"📊 55-Day Bullish Breakout + FVG + FVG Test: {res['Stock']}")
        print("----------------------------")
        for k, v in res.items():
            if k != "Stock":
                print(f"{k}: {v}")
    # Save CSV
    pd.DataFrame(df_results).to_csv("nasdaq_fvg_bullish_breakouts_55d.csv", index=False)
else:
    print("⚠️ No valid bullish breakouts with meaningful FVG found in the last 30 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

⚠️ No valid bullish breakouts with meaningful FVG found in the last 30 days.


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 10  # recent 10-day highs
today = datetime.today().date()
start_date = today - timedelta(days=55 + lookback_days + 30)  # buffer

tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]

df_results = []

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):
        B = df.iloc[i]
        A = df.iloc[i-1]
        C = df.iloc[i+1]

        B_close = B['Close'].item()
        B_open = B['Open'].item()
        A_high = A['High'].item()
        C_low = C['Low'].item()
        date = df.index[i].date()

        # 55-day high before breakout
        prev_55d_slice = df['High'].iloc[max(i-55,0):i]
        if prev_55d_slice.empty:
            continue
        prev_55d_high = prev_55d_slice.max()
        if isinstance(prev_55d_high, pd.Series):
            prev_55d_high = prev_55d_high.item()

        # Recent 10-day high
        recent_10d_slice = df['High'].iloc[max(i-10,0):i]
        recent_10d_high = recent_10d_slice.max()
        if isinstance(recent_10d_high, pd.Series):
            recent_10d_high = recent_10d_high.item()

        # Breakout conditions
        if (
            B_close > prev_55d_high and
            B_close > B_open and
            recent_10d_high < prev_55d_high
        ):
            # FVG condition
            if C_low > A_high:
                fvg_low = A_high
                fvg_high = C_low
                fvg_date = date

                # Look forward for FVG test
                test_date = None
                test_low = None
                for j in range(i+1, total_len):
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break

                df_results.append({
                    "Stock": ticker,
                    "Breakout Date": fvg_date,
                    "Prev 55D High": prev_55d_high,
                    "Recent 10-Day High": recent_10d_high,
                    "Breakout Close": B_close,
                    "Candle A (Date)": A.name.date(),
                    "Candle A High": A_high,
                    "Candle C (Date)": C.name.date(),
                    "Candle C Low": C_low,
                    "FVG Zone": f"{A_high} → {C_low}",
                    "FVG First Test Date": test_date,
                    "FVG First Test Low": test_low
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=False)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Print Latest FVG per stock
# ---------------------------
if df_results:
    df_results_df = pd.DataFrame(df_results)
    latest_fvg_df = df_results_df.sort_values("Breakout Date").groupby("Stock").tail(1)

    for idx, res in latest_fvg_df.iterrows():
        print("\n============================")
        print(f"📊 Latest FVG Zone: {res['Stock']}")
        print("----------------------------")
        for k, v in res.items():
            if k != "Stock":
                print(f"{k}: {v}")
else:
    print("⚠️ No valid bullish breakouts with meaningful FVG found in the last 10 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


📊 Latest FVG Zone: AAPL
----------------------------
Breakout Date: 2025-09-03
Prev 55D High: 235.1199951171875
Recent 10-Day High: 233.41000366210938
Breakout Close: 238.47000122070312
Candle A (Date): 2025-09-02
Candle A High: 230.85000610351562
Candle C (Date): 2025-09-04
Candle C Low: 236.74000549316406
FVG Zone: 230.85000610351562 → 236.74000549316406
FVG First Test Date: 2025-09-04
FVG First Test Low: 236.74000549316406


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 10  # recent 10-day high
today = datetime.today().date()
start_date = today - timedelta(days=55 + lookback_days + 30)  # buffer for 55D + 10D



tickers = ['AACB', 'AACBU', 'AACI', 'AACIU', 'AADR', 'AAL', 'AALG', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPG', 'AAPL', 'AAPU', 'AARD', 'AAUS', 'AAVM', 'AAXJ', 'ABCS', 'ABI', 'ABIG', 'ABLLL', 'ABNB', 'ABVX', 'ACAD', 'ACFN', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACIC', 'ACIW', 'ACLS', 'ACLX', 'ACMR', 'ACNB', 'ACNT', 'ACT', 'ACWI', 'ACWX', 'ADBG', 'ADEA', 'ADI', 'ADMA', 'ADP', 'ADPT', 'ADUR', 'ADUS', 'AEBI', 'AEHR', 'AEIS', 'AEP', 'AEVA', 'AEYE', 'AFBI', 'AFJK', 'AFJKU', 'AFOS', 'AFRM', 'AFSC', 'AFYA', 'AGEM', 'AGGA', 'AGIO', 'AGIX', 'AGMH', 'AGMI', 'AGNCL', 'AGNCM', 'AGNCN', 'AGNCO', 'AGNCP', 'AGNG', 'AGYS', 'AGZD', 'AIA', 'AIFD', 'AIPI', 'AIPO', 'AIQ', 'AIRL', 'AIRO', 'AIRR', 'AIRT', 'AIRTP', 'AKAM', 'AKRO', 'ALAB', 'ALAR', 'ALCO', 'ALCY', 'ALCYU', 'ALDF', 'ALDFU', 'ALF', 'ALFUU', 'ALGM', 'ALGN', 'ALGT', 'ALHC', 'ALIL', 'ALKS', 'ALKT', 'ALLW', 'ALMU', 'ALNT', 'ALOT', 'ALRM', 'ALRS', 'ALTY', 'AMAL', 'AMAT', 'AMBA', 'AMD', 'AMDD', 'AMDG', 'AMDU', 'AMGN', 'AMID', 'AMKR', 'AMLX', 'AMPH', 'AMPL', 'AMRK', 'AMRN', 'AMSC', 'AMSF', 'AMUU', 'AMWD', 'AMZD', 'AMZN', 'AMZU', 'AMZZ', 'ANAB', 'ANDE', 'ANGI', 'ANGL', 'ANGO', 'ANIP', 'ANPA', 'ANSC', 'ANSCU', 'ANTA', 'AOHY', 'AOSL', 'AOTG', 'APA', 'APADU', 'APED', 'APEI', 'APGE', 'APLD', 'APLM', 'APLS', 'APOG', 'APPF', 'APPN', 'APPX', 'AQWA', 'ARCB', 'ARCC', 'ARCT', 'ARHS', 'ARLP', 'ARM', 'ARMG', 'AROW', 'ARQQ', 'ARQT', 'ARTNA', 'ARVR', 'ARWR', 'ASET', 'ASMB', 'ASMG', 'ASND', 'ASO', 'ASPC', 'ASPCU', 'ASPS', 'ASTE', 'ASTH', 'ASTS', 'ATAT', 'ATEC', 'ATEX', 'ATGL', 'ATII', 'ATIIU', 'ATLC', 'ATLCL', 'ATLCP', 'ATLCZ', 'ATLO', 'ATMC', 'ATMCU', 'ATMV', 'ATMVU', 'ATNI', 'ATRA', 'ATRC', 'ATRO', 'AUBN', 'AUDC', 'AUGO', 'AUMI', 'AUPH', 'AVAV', 'AVBH', 'AVBP', 'AVDL', 'AVGB', 'AVGG', 'AVGU', 'AVGX', 'AVL', 'AVNW', 'AVO', 'AVPT', 'AVS', 'AVT', 'AVTX', 'AVUQ', 'AVXC', 'AXGN', 'AXINU', 'AXSM', 'AZN', 'AZTA']

tickers1 = ['BABX', 'BACCU', 'BACQ', 'BACQU', 'BAFE', 'BAIG', 'BAND', 'BANF', 'BANFP', 'BANR', 'BANX', 'BASE', 'BASG', 'BASV', 'BATRA', 'BATRK', 'BAYA', 'BAYAU', 'BBH', 'BBIO', 'BBLGW', 'BBNX', 'BBOT', 'BBSI', 'BCAL', 'BCARU', 'BCAX', 'BCIC', 'BCLO', 'BCML', 'BCPC', 'BDGS', 'BEAG', 'BEAGU', 'BEAM', 'BEEX', 'BEEZ', 'BELFA', 'BELFB', 'BELT', 'BETR', 'BFC', 'BFIN', 'BFST', 'BGL', 'BGRN', 'BGRO', 'BHF', 'BHFAL', 'BHFAM', 'BHFAN', 'BHFAO', 'BHFAP', 'BHRB', 'BIB', 'BIDU', 'BIIB', 'BILI', 'BIS', 'BITS', 'BJK', 'BJRI', 'BKCH', 'BKHA', 'BKHAU', 'BKIV', 'BKR', 'BKWO', 'BL', 'BLBD', 'BLCN', 'BLCR', 'BLFS', 'BLKB', 'BLTE', 'BLUWU', 'BMAX', 'BMDL', 'BMRC', 'BMRN', 'BND', 'BNDW', 'BNDX', 'BNTC', 'BNTX', 'BOED', 'BOEG', 'BOEU', 'BOKF', 'BOTJ', 'BOTT', 'BOTZ', 'BOWNU', 'BPOP', 'BPOPM', 'BPRN', 'BPYPM', 'BPYPN', 'BPYPO', 'BPYPP', 'BREA', 'BRHY', 'BRKD', 'BRKR', 'BRKU', 'BRNY', 'BRR', 'BRRR', 'BRRWU', 'BRTR', 'BRZE', 'BSAA', 'BSAAU', 'BSCP', 'BSCQ', 'BSCR', 'BSCS', 'BSCT', 'BSCU', 'BSCV', 'BSCW', 'BSCX', 'BSCY', 'BSCZ', 'BSET', 'BSJP', 'BSJQ', 'BSJR', 'BSJS', 'BSJT', 'BSJU', 'BSJV', 'BSJW', 'BSJX', 'BSMP', 'BSMQ', 'BSMR', 'BSMS', 'BSMT', 'BSMU', 'BSMV', 'BSMW', 'BSMY', 'BSRR', 'BSSX', 'BSVN', 'BSVO', 'BSY', 'BTDR', 'BTF', 'BTFX', 'BTGD', 'BTSG', 'BTSGU', 'BUFC', 'BUFI', 'BUFM', 'BUG', 'BULD', 'BULG', 'BULL', 'BUSE', 'BUSEP', 'BVFL', 'BWAY', 'BWB', 'BWBBP', 'BWFG', 'BWIN', 'BWMN', 'BYRN', 'BZ', 'CA', 'CAC', 'CAEP', 'CAFG', 'CAI', 'CAKE', 'CALI', 'CALM', 'CAMT', 'CANC', 'CANQ', 'CAPN', 'CAPNU', 'CAR', 'CARE', 'CARG', 'CARL', 'CART', 'CARY', 'CARZ', 'CASH', 'CASS', 'CATH', 'CATY', 'CBFV', 'CBIO', 'CBLL', 'CBNK', 'CBRL', 'CBSH', 'CCAP', 'CCB', 'CCBG', 'CCCX', 'CCCXU', 'CCD', 'CCEC', 'CCEP', 'CCFE', 'CCII', 'CCIIU', 'CCIX', 'CCIXU', 'CCLDO', 'CCNE', 'CCNEP', 'CCNR', 'CCOI', 'CCRN', 'CCSB', 'CCSI', 'CCSO', 'CDC', 'CDL', 'CDNA', 'CDTX', 'CDW', 'CDZIP', 'CECO', 'CEFA', 'CELC', 'CELH', 'CENT', 'CENTA', 'CENX', 'CEP', 'CEPF', 'CEPI', 'CEPO', 'CEPT', 'CERT', 'CEVA', 'CFA', 'CFBK', 'CFFI', 'CFLT', 'CFO', 'CFSB', 'CG', 'CGABL', 'CGBD', 'CGBDL', 'CGCT', 'CGCTU', 'CGNX', 'CGO', 'CGON', 'CHA', 'CHAC', 'CHACU', 'CHAR', 'CHARU', 'CHCI', 'CHCO', 'CHDN', 'CHEF', 'CHGX', 'CHI', 'CHKP', 'CHMG', 'CHPG', 'CHPGU', 'CHPS', 'CHRD', 'CHRW', 'CHSCL', 'CHSCM', 'CHSCN', 'CHSCO', 'CHSCP', 'CHTR', 'CHY', 'CHYM', 'CIBR', 'CIFR', 'CIGI', 'CIL', 'CINF', 'CIVB', 'CLBK', 'CLBT', 'CLDX', 'CLFD', 'CLMB', 'CLMT', 'CLOA', 'CLOD', 'CLOU', 'CLPT', 'CLSK', 'CLSM', 'CLST', 'CMCO', 'CMCSA', 'CME', 'CMPO', 'CMPOW', 'CMPR', 'CNFRZ', 'CNOB', 'CNOBP', 'CNTA', 'CNXC', 'CNXN', 'COCO', 'COEP', 'COFS', 'COGT', 'COHU', 'COIG', 'COKE', 'COLA', 'COLAU', 'COLB', 'COLL', 'COLM', 'COMM', 'COMT', 'CONI', 'CONL', 'COO', 'COOP', 'COPJ', 'COPP', 'CORO', 'CORT', 'CORZ', 'CORZW', 'CORZZ', 'COWG', 'COWS', 'COYY', 'CPAG', 'CPB', 'CPBI', 'CPHC', 'CPHY', 'CPLS', 'CPRT', 'CPRX', 'CPZ', 'CRAI', 'CRAQU', 'CRBP', 'CRCG', 'CRDO', 'CRMD', 'CRMG', 'CRMT', 'CRNC', 'CRNX', 'CROX', 'CRSP', 'CRTO', 'CRUS', 'CRVL', 'CRWG', 'CRWL', 'CRWV', 'CSB', 'CSCL', 'CSCO', 'CSCS', 'CSGP', 'CSGS', 'CSIQ', 'CSPI', 'CSQ', 'CSTL', 'CSWC', 'CSWCZ', 'CSX', 'CTAS', 'CTBI', 'CTEC', 'CTLP', 'CTNM', 'CTRN', 'CTSH', 'CUB', 'CUBWU', 'CVBF', 'CVGW', 'CVKD', 'CVLT', 'CVNX', 'CWBC', 'CWCO', 'CWST', 'CXSE', 'CYTK', 'CZAR', 'CZFS', 'CZNC', 'CZR', 'CZWI', 'DAAQ', 'DAAQU', 'DADS', 'DAK', 'DAKT', 'DALI', 'DALN', 'DAPP', 'DASH', 'DAVE', 'DAX', 'DBX', 'DCBO', 'DCOM', 'DCOMG', 'DCOMP', 'DCTH', 'DDIV', 'DDOG', 'DECO', 'DEMZ', 'DFDV', 'DFGP', 'DFGX', 'DGCB', 'DGICA', 'DGICB', 'DGII', 'DGLO', 'DGNX', 'DGRE', 'DGRS', 'DGRW', 'DHCNI', 'DHCNL', 'DHIL', 'DIOD', 'DIVD', 'DJT', 'DKNG', 'DKNX', 'DLLL', 'DLO', 'DLTR', 'DMAA', 'DMAAU', 'DMAT', 'DMLP', 'DMRC', 'DMXF', 'DNLI', 'DNTH', 'DOCU', 'DOGZ', 'DOMO', 'DOOO', 'DORM', 'DOX', 'DRDB', 'DRDBU', 'DRIV', 'DRS', 'DRUG', 'DRVN', 'DSGR', 'DSGX', 'DTCR', 'DTSQ', 'DTSQU', 'DUKH', 'DUKX', 'DUOL', 'DVAL', 'DVIN', 'DVLU', 'DVOL', 'DVQQ', 'DVRE', 'DVSP', 'DVUT', 'DVXB', 'DVXC', 'DVXE', 'DVXF', 'DVXK', 'DVXP','DVIN', 'DVLU', 'DVOL', 'DVQQ', 'DVRE', 'DVSP', 'DVUT', 'DVXB', 'DVXC', 'DVXE', 'DVXF', 'DVXK', 'DVXP', 'DVXV', 'DVXY', 'DVY', 'DWAS', 'DWAW', 'DWUS', 'DXCM', 'DXPE', 'DXR', 'DYCQ', 'DYCQU', 'DYFI', 'DYN', 'DYTA', 'EA', 'EBAY', 'EBC', 'EBI', 'EBIZ', 'EBMT', 'ECBK', 'ECOW', 'ECPG', 'EDRY', 'EEFT', 'EEMA', 'EFAS', 'EFRA', 'EFSC', 'EFSCP', 'EFSI', 'EGBN', 'EGGQ', 'EGHA', 'EGHAU', 'EH', 'EHLS', 'EKG', 'ELFY', 'ELIL', 'ELIS', 'ELTK', 'ELTX', 'ELVN', 'EMB', 'EMBC', 'EMCB', 'EMEQ', 'EMIF', 'EML', 'EMPG', 'EMXC', 'EMXF', 'ENDW', 'ENLT', 'ENPH', 'ENSG', 'ENTG', 'ENVX', 'ENZL', 'EPSM', 'EQRR', 'ERET', 'ERII', 'ERNZ', 'ESCA', 'ESEA', 'ESGD', 'ESGE', 'ESGU', 'ESHA', 'ESMV', 'ESN', 'ESOA', 'ESPO', 'ESQ', 'ESTA', 'ETEC', 'ETHA', 'ETHM', 'ETHMU', 'ETNB', 'ETON', 'ETOR', 'ETSY', 'EUFN', 'EURK', 'EURKU', 'EVCM', 'EVER', 'EVMT', 'EVRG', 'EVSD', 'EVYM', 'EWBC', 'EWJV', 'EWTX', 'EWZS', 'EXAS', 'EXC', 'EXE', 'EXEEL', 'EXEEW', 'EXEEZ', 'EXEL', 'EXLS', 'EXOZ', 'EXPE', 'EXPI', 'EXPO', 'EXTR', 'EXUS', 'EYE', 'EYEG', 'EYPT', 'EZPW', 'FA', 'FAAR', 'FAB', 'FACT', 'FACTU', 'FAD', 'FALN', 'FANG', 'FAST', 'FBIZ', 'FBL', 'FBLA', 'FBNC', 'FBOT', 'FBRX', 'FBYD', 'FCA', 'FCAL', 'FCAP', 'FCBC', 'FCCO', 'FCEF', 'FCFS', 'FCNCO', 'FCNCP', 'FCTE', 'FCVT', 'FDBC', 'FDCF', 'FDFF', 'FDIF', 'FDIG', 'FDIV', 'FDNI', 'FDSB', 'FDT', 'FDTS', 'FDTX', 'FDUS', 'FEAT', 'FEIM', 'FELE', 'FEM', 'FEMB', 'FEMS', 'FEP', 'FEPI', 'FER', 'FERA', 'FERAU', 'FEUZ', 'FEX', 'FFBC', 'FFIC', 'FFIN', 'FFUT', 'FGBIP', 'FGEN', 'FGM', 'FGMCU', 'FGNXP', 'FGSI', 'FHB', 'FIBK', 'FICS', 'FID', 'FIGXU', 'FINE', 'FINW', 'FINX', 'FISI', 'FITB', 'FITBI', 'FITBO', 'FITBP', 'FIVE', 'FIVN', 'FIVY', 'FIXD', 'FIZZ', 'FJP', 'FKU', 'FLDB', 'FLEX', 'FLGC', 'FLGT', 'FLN', 'FLXS', 'FLY', 'FLYW', 'FMAO', 'FMB', 'FMBH', 'FMED', 'FMET', 'FMFC', 'FMHI', 'FMNB', 'FMTM', 'FMUB', 'FMUN', 'FNK', 'FNLC', 'FNWD', 'FNX', 'FNY', 'FONR', 'FORD', 'FORL', 'FORLU', 'FORM', 'FORR', 'FORTY', 'FOSLL', 'FOX', 'FOXA', 'FOXF', 'FPA', 'FPXE', 'FPXI', 'FRAF', 'FRBA', 'FRD', 'FRDD', 'FRDU', 'FRHC', 'FRME', 'FRMEP', 'FROG', 'FRPH', 'FRPT', 'FRSH', 'FRST', 'FSBC', 'FSBW', 'FSCS', 'FSEA', 'FSFG', 'FSGS', 'FSHP', 'FSHPU', 'FSLR', 'FSTR', 'FSUN', 'FSV', 'FSZ', 'FTA', 'FTAG', 'FTAI', 'FTAIM', 'FTAIN', 'FTC', 'FTCS', 'FTDR', 'FTDS', 'FTGC', 'FTGS', 'FTHI', 'FTLF', 'FTNT', 'FTQI', 'FTRE', 'FTRI', 'FTSL', 'FTSM', 'FTXG', 'FTXH', 'FTXL', 'FTXN', 'FTXO', 'FTXR', 'FULT', 'FULTP', 'FUNC', 'FUSB', 'FUTU', 'FV', 'FVC', 'FVCB', 'FVN', 'FVNNU', 'FWONA', 'FWONK', 'FWRD', 'FWRG', 'FXNC', 'FYBR', 'FYC', 'FYT', 'FYX', 'GABC', 'GAIN', 'GAINI', 'GAINL', 'GAINN', 'GAINZ', 'GBDC', 'GBFH', 'GBUG', 'GCBC', 'GCMG', 'GCT', 'GDEN', 'GDEV', 'GDS', 'GECC', 'GECCH', 'GECCI', 'GECCO', 'GECCZ', 'GEGGL', 'GEHC', 'GEME', 'GEN', 'GEOS', 'GFGF', 'GFLW', 'GFS', 'GGAL', 'GGLL', 'GH', 'GHRS', 'GIG', 'GIGGU', 'GIII', 'GILD', 'GILT', 'GIND', 'GINX', 'GKAT', 'GLAD', 'GLADZ', 'GLBE', 'GLCR', 'GLDD', 'GLDI', 'GLDY', 'GLGG', 'GLIBA', 'GLIBK', 'GLNG', 'GLOW', 'GLPG', 'GLPI', 'GLRE', 'GLSI', 'GLXY', 'GMAB', 'GNMA', 'GNOM', 'GNTX', 'GO', 'GOOD', 'GOODN', 'GOODO', 'GOOG', 'GOOGL', 'GOVI', 'GPAT', 'GPATU', 'GPCR', 'GPIQ', 'GPIX', 'GPRF', 'GQQQ', 'GRAL', 'GRID', 'GRIN', 'GRPN', 'GRRR', 'GRVY', 'GRW', 'GSAT', 'GSBC', 'GSHD', 'GSHR', 'GSHRU','GSIB', 'GSRT', 'GSRTU', 'GTEN', 'GTENU', 'GTERU', 'GTLB', 'GTM', 'GTR', 'GTX', 'GXDW', 'GYRO', 'HAFC', 'HALO', 'HAS', 'HBAN', 'HBANL', 'HBANM', 'HBANP', 'HBCP', 'HBDC', 'HBNC', 'HBT', 'HCKT', 'HCM', 'HCMAU', 'HCOW', 'HCSG', 'HDL', 'HEAL', 'HECO', 'HEJD', 'HELE', 'HEQQ', 'HERD', 'HERO', 'HFBL', 'HFSP', 'HFWA', 'HIDE', 'HIFS', 'HIMY', 'HIMZ', 'HISF', 'HLAL', 'HLIT', 'HLNE', 'HNDL', 'HNNA', 'HNNAZ', 'HNRG', 'HNVR', 'HOFT', 'HOLX', 'HON', 'HOND', 'HONDU', 'HONE', 'HOOD', 'HOOG', 'HOOI', 'HOOX', 'HOPE', 'HOVNP', 'HQGO', 'HQY', 'HRMY', 'HROW', 'HROWL', 'HROWM', 'HRTS', 'HSAI', 'HSDT', 'HSIC', 'HSII', 'HSPO', 'HSPOU', 'HSPT', 'HSPTU', 'HST', 'HSTM', 'HTBK', 'HTFL', 'HTHT', 'HTO', 'HUBG', 'HURC', 'HURN', 'HUT', 'HVII', 'HVIIU', 'HVMCU', 'HWAY', 'HWBK', 'HWC', 'HWCPZ', 'HWKN', 'HWSM', 'HYBI', 'HYDR', 'HYLS', 'HYPD', 'HYXF', 'HYZD', 'IAC', 'IART', 'IBAC', 'IBAT', 'IBB', 'IBBQ', 'IBCP', 'IBEX', 'IBGA', 'IBGB', 'IBGK', 'IBGL', 'IBIT', 'IBKR']

tickers2 = ['IBOC', 'IBOT', 'IBTF', 'IBTG', 'IBTH', 'IBTI', 'IBTJ', 'IBTK', 'IBTL', 'IBTM', 'IBTO', 'IBTP', 'IBTQ', 'ICFI', 'ICHR', 'ICLN', 'ICLR', 'ICOP', 'ICUI', 'IDEF', 'IDYA', 'IEF', 'IEI', 'IEUS', 'IFGL', 'IFLO', 'IFV', 'IGF', 'IGIB', 'IGIC', 'IGOV', 'IGSB', 'IHYF', 'IIIV', 'IJT', 'ILIT', 'ILMN', 'IMCR', 'IMCV', 'IMKTA', 'IMOM', 'IMOS', 'IMPPP', 'IMVT', 'IMXI', 'INACU', 'INBK', 'INBKZ', 'INBX', 'INCY', 'INDB', 'INDH', 'INDV', 'INDY', 'INFR', 'INKT', 'INMD', 'INOD', 'INRO', 'INSG', 'INSM', 'INTA', 'INTC', 'INTG', 'INTW', 'INVA', 'IONL', 'IONS', 'IONX', 'IOSP', 'IPAR', 'IPCX', 'IPCXU', 'IPGP', 'IPKW', 'IPOD', 'IPODU', 'IPX', 'IQQQ', 'IRDM', 'IREN', 'IRMD', 'IROH', 'IRON', 'IROQ', 'IRTC', 'ISBA', 'ISHG', 'ISHP', 'ISRL', 'ISRLU', 'ISSC', 'ISTB', 'ISTR', 'ITIC', 'ITRI', 'ITRN', 'IUS', 'IUSB', 'IUSG', 'IUSV', 'IVAL', 'IXUS', 'JACK', 'JAKK', 'JAMF', 'JANX', 'JAPN', 'JAZZ', 'JBHT', 'JBSS', 'JCAP', 'JD', 'JDOC', 'JEPQ', 'JFIN', 'JGLO', 'JHAI', 'JIVE', 'JJSF', 'JKHY', 'JMID', 'JMSB', 'JOUT', 'JOYY', 'JPEF', 'JPX', 'JPY', 'JSM', 'JSMD', 'JSML', 'JTEK', 'KALA', 'KALU', 'KALV', 'KARO', 'KAT', 'KBAB', 'KBWB', 'KBWD', 'KBWP', 'KBWR', 'KBWY', 'KC', 'KCHV', 'KCHVU', 'KDP', 'KE', 'KEAT', 'KELYA', 'KELYB', 'KEQU', 'KFII', 'KFIIU', 'KG', 'KHC', 'KIDS', 'KINS', 'KLIC', 'KMB', 'KMLI', 'KMTS', 'KNGZ', 'KNSA', 'KOID', 'KOYNU', 'KPDD', 'KPLT', 'KQQQ', 'KRMA', 'KRNT', 'KROP', 'KROS', 'KRRO', 'KRT', 'KRUS', 'KRYS', 'KSPI', 'KTOS', 'KVAC', 'KVACU', 'KYIV', 'KYMR', 'LAKE', 'LAMR', 'LANDM', 'LANDO', 'LANDP', 'LARK', 'LASR', 'LAUR', 'LAYS', 'LBRDA', 'LBRDK', 'LBRDP', 'LBTYA', 'LBTYB', 'LBTYK', 'LCCC', 'LCCCU', 'LCDL', 'LCDS', 'LCFYW', 'LCID', 'LCNB', 'LDEM', 'LDRX', 'LDSF', 'LE', 'LECO', 'LEGH', 'LEGN', 'LEGR', 'LENZ', 'LEXI', 'LFMDP', 'LFSC', 'LFUS', 'LFVN', 'LGCF', 'LGIH', 'LGND', 'LGRO', 'LI', 'LIEN', 'LIF', 'LINC', 'LIND', 'LINE', 'LINK', 'LITE', 'LIVE', 'LIVN', 'LIVR', 'LKFN', 'LKQ', 'LLYVA', 'LLYVK', 'LMAT', 'LMB', 'LMBS', 'LMNR', 'LMTL', 'LMTS', 'LNSR', 'LNT', 'LNTH', 'LNW', 'LNZA', 'LOGI', 'LOGO', 'LOKV', 'LOKVU', 'LOPE', 'LOVE', 'LPAA', 'LPAAU', 'LPBB', 'LPBBU', 'LQDA', 'LQDT', 'LRCX', 'LRGE', 'LRND', 'LSBK', 'LSCC', 'LSTR', 'LTBR', 'LULU', 'LVHD', 'LWACU', 'LWAY', 'LYEL', 'LYFT', 'LYTS', 'LZ', 'MACI', 'MACIU', 'MAMA', 'MANH', 'MAR', 'MARA', 'MASI', 'MAT', 'MATW', 'MAXI', 'MAYA', 'MAYAU', 'MAYS', 'MAZE', 'MBAV', 'MBAVU', 'MBB', 'MBBC', 'MBCN', 'MBIN', 'MBINL', 'MBINM', 'MBINN', 'MBLY', 'MBNKO', 'MBUU', 'MBVIU', 'MBWM', 'MCBS', 'MCDS', 'MCFT', 'MCHI', 'MCHP', 'MCHPP', 'MCHS', 'MCRB', 'MCRI', 'MCSE', 'MDIV', 'MDLZ', 'MDRR', 'MDWD', 'MEDX', 'MEMS', 'MENS', 'MEOH', 'MESO', 'METC', 'METCB', 'METCI', 'METCZ', 'METD', 'METU', 'MFI', 'MFIC', 'MFICL', 'MFIN', 'MFLX', 'MGEE', 'MGIC', 'MGNI', 'MGPI', 'MGRC', 'MGYR', 'MIDD', 'MILN', 'MIRM', 'MITK', 'MKAM', 'MKSI', 'MKTW', 'MKTX', 'MLAB', 'MLAC', 'MLACU', 'MLGO', 'MLKN', 'MLTX', 'MLYS', 'MMSI', 'MMYT', 'MNDY', 'MNPR', 'MNRO', 'MNSB', 'MNSBP', 'MNST', 'MODL', 'MOFG', 'MOOD', 'MORN', 'MPAA', 'MPB', 'MQQQ', 'MRAL', 'MRBK', 'MRCY', 'MRNA', 'MRTN', 'MRUS', 'MRVL', 'MRX', 'MSBI', 'MSBIP', 'MSDD', 'MSEX', 'MSFD', 'MSFL', 'MSFU', 'MSGY', 'MST', 'MSTP', 'MSTX', 'MTCH', 'MTEN', 'MTRX', 'MTSI', 'MTSR', 'MU', 'MUD', 'MULL', 'MULT', 'MUU', 'MVBF', 'MVLL', 'MXL', 'MYCF', 'MYCG', 'MYCH', 'MYCI', 'MYCJ', 'MYCK', 'MYCL', 'MYCM', 'MYCN', 'MYFW', 'MYMF', 'MYMG', 'MYMH', 'MYMI', 'MYMJ', 'MYRG', 'MZTI', 'NAMS', 'NAMSW', 'NATH', 'NATO', 'NATR', 'NAVI', 'NBBK', 'NBIS', 'NBIX', 'NBN', 'NBTB', 'NBTX', 'NCIQ', 'NCNO', 'NCPB', 'NCSM', 'NDAA', 'NDAQ', 'NDSN', 'NECB', 'NEGG', 'NERD', 'NESR', 'NETD', 'NETDU', 'NEUP', 'NEWT', 'NEWTG', 'NEWTH', 'NEWTI', 'NEWTP', 'NEWTZ', 'NEWZ', 'NFBK', 'NFTY', 'NFXL', 'NFXS', 'NGNE', 'NHIC', 'NHICU', 'NHPAP', 'NHPBP', 'NICE', 'NIKL', 'NIXT', 'NKSH', 'NKTR', 'NMFCZ', 'NMIH', 'NMPAU', 'NMRK', 'NN', 'NNE', 'NNNN', 'NODK', 'NOEM', 'NOEMU', 'NOVT', 'NOWL', 'NPAC', 'NPACU', 'NPFI', 'NRC', 'NRDS', 'NRES', 'NRIM', 'NSCR', 'NSI', 'NSIT', 'NSSC', 'NSTS', 'NTAP', 'NTCT', 'NTES', 'NTGR', 'NTHI', 'NTLA', 'NTNX', 'NTRA', 'NTRS', 'NTRSO', 'NTWO', 'NTWOU', 'NUSB', 'NUTX', 'NUVL', 'NVA', 'NVCR', 'NVDA', 'NVDG', 'NVDL', 'NVDS', 'NVDU', 'NVEC', 'NVYY', 'NWBI', 'NWE', 'NWFL', 'NWPX', 'NWS', 'NWSA', 'NXPI', 'NXST', 'NXT', 'NXTG', 'NXTT', 'NYAX', 'NYMTI', 'NZAC', 'NZUS', 'OACC', 'OACCU', 'OAKU', 'OAKUU', 'OBIL', 'OBT', 'OCCIM', 'OCCIN', 'OCCIO', 'OCFC', 'OCS', 'OCSL', 'OCUL', 'ODD', 'ODDS', 'ODFL', 'ODP', 'OFIX', 'OFLX', 'OFSSH', 'OFSSO', 'OKLL', 'OKTA', 'OLED', 'OLLI', 'OM', 'OMAB', 'OMCL', 'OMDA', 'ON', 'ONB', 'ONBPO', 'ONBPP', 'ONCHU', 'ONEG', 'ONEQ', 'ONEW', 'OOQB', 'OOSB', 'OPBK', 'OPCH', 'OPPJ', 'OPRA', 'OPRX', 'OPTZ', 'OPXS', 'ORCX', 'ORIC', 'ORIQU', 'ORKA', 'ORLY', 'ORR', 'ORRF', 'OS', 'OSBC', 'OSIS', 'OSPN', 'OSW', 'OTEX', 'OTLY', 'OTTR', 'OUST', 'OVBC', 'OVLY', 'OXLC', 'OXLCG', 'OXLCI', 'OXLCL', 'OXLCN', 'OXLCO', 'OXLCP', 'OXLCZ', 'OXSQG', 'OXSQH', 'OXSQZ', 'OYSE', 'OYSEU', 'OZEM', 'OZK', 'OZKAP', 'PAA', 'PABD', 'PABU', 'PACHU', 'PAGP', 'PAHC', 'PALD', 'PALU', 'PAMT', 'PANG', 'PANW', 'PATK', 'PATN', 'PAX', 'PAYX', 'PBBK', 'PBFS', 'PBHC', 'PBPB', 'PBQQ', 'PCAP', 'PCAPU', 'PCAR', 'PCB', 'PCH', 'PCMM', 'PCRX', 'PCSC', 'PCT', 'PCTTU', 'PCTY', 'PCVX', 'PCYO', 'PDBA', 'PDBC', 'PDD', 'PDDL', 'PDEX', 'PDFS', 'PDLB', 'PDP', 'PEBK', 'PEBO', 'PECO', 'PEGA', 'PELI', 'PELIU', 'PENG', 'PENN', 'PEP', 'PEPS', 'PEY', 'PEZ', 'PFBC', 'PFF', 'PFG', 'PFI', 'PFIS', 'PFM', 'PFX', 'PFXNZ', 'PGAC', 'PGACU', 'PGC', 'PGJ', 'PGNY', 'PGY', 'PHAR', 'PHAT', 'PHO', 'PHVS', 'PI', 'PID', 'PIE', 'PINC', 'PIO', 'PIZ', 'PKBK', 'PKOH', 'PKW', 'PLAB', 'PLAY', 'PLBC', 'PLMK', 'PLMKU', 'PLMR', 'PLPC', 'PLSE', 'PLT', 'PLTG', 'PLTR', 'PLTU', 'PLUS', 'PLXS', 'PMBS', 'PMTR', 'PMTRU', 'PMTS', 'PNFP', 'PNFPP', 'PNQI', 'PNRG', 'PNTG', 'POLE', 'POLEU', 'PONY', 'POWI', 'POWL', 'POWWP', 'PPC', 'PPH', 'PPI', 'PPIH', 'PPTA', 'PQAP', 'PQJA', 'PQJL', 'PQOC', 'PRAA', 'PRAX', 'PRCH', 'PRCT', 'PRDO', 'PRE', 'PRFZ', 'PRGS', 'PRN', 'PROV', 'PRPO', 'PRTC', 'PRVA', 'PSC', 'PSCC', 'PSCD', 'PSCE', 'PSCF', 'PSCH', 'PSCI', 'PSCM', 'PSCT', 'PSCU', 'PSET', 'PSIX', 'PSKY', 'PSL', 'PSMT', 'PSTR', 'PSWD', 'PTC', 'PTCT', 'PTF', 'PTGX', 'PTH', 'PTIR', 'PTNQ', 'PUI', 'PVBC', 'PVLA', 'PWP', 'PWRD', 'PXI', 'PXLW', 'PY', 'PYPG', 'PYPL', 'PYZ', 'PZZA', 'QABA', 'QALT', 'QAT', 'QB', 'QBIG', 'QBUF', 'QCLN', 'QCLR', 'QCMD', 'QCML', 'QCMU', 'QCOM', 'QCRH', 'QDEL', 'QDTY', 'QETA', 'QETAU', 'QFIN', 'QGRD', 'QHDG', 'QLYS', 'QMCO', 'QMID', 'QMMM', 'QMOM', 'QNST', 'QNTM', 'QNXT', 'QOWZ', 'QPUX', 'QQA', 'QQDN', 'QQEW', 'QQHG', 'QQJG', 'QQLV', 'QQMG', 'QQQA', 'QQQE', 'QQQG', 'QQQH', 'QQQI', 'QQQJ', 'QQQM', 'QQQP', 'QQQS', 'QQQT', 'QQQX', 'QQQY', 'QQUP', 'QQWZ', 'QQXL', 'QQXT', 'QRMI', 'QRVO', 'QSEA', 'QSEAU', 'QSIX', 'QSML', 'QTEC', 'QTOP', 'QTR', 'QTUM', 'QUBT', 'QUMSU', 'QURE', 'QVAL', 'QVCGA', 'QXQ', 'QYLD', 'QYLG', 'RAA', 'RAAQ', 'RAAQU', 'RAND', 'RANG', 'RANGU', 'RAPP', 'RAPT', 'RARE', 'RBB', 'RBCAA', 'RBIL', 'RBKB', 'RCAT', 'RCGE', 'RCKY', 'RCMT', 'RDAC', 'RDACU', 'RDAG', 'RDAGU', 'RDCM', 'RDIB', 'RDNT', 'RDTL', 'RDTY', 'RDVT', 'RDVY', 'RDWR', 'REAI', 'REAL', 'REFI', 'REG', 'REGCO', 'REGCP', 'REIT', 'RELL', 'RELY', 'REMG', 'REYN', 'RFAI', 'RFAIU', 'RFDI', 'RFEM', 'RFEU', 'RGC', 'RGCO', 'RGEN', 'RGLD', 'RGLO', 'RGS', 'RGTI', 'RGTIW', 'RGTX', 'RHLD', 'RIBB', 'RIBBU', 'RICK', 'RIFR', 'RIGL', 'RILYG', 'RILYK', 'RILYN', 'RILYT', 'RILYZ', 'RING', 'RINT', 'RIOT', 'RIVN', 'RKLB', 'RKLX', 'RMBI', 'RMBS', 'RMR', 'RNA', 'RNAZ', 'RNEM', 'RNIN', 'RNRG', 'ROAD', 'ROBT', 'ROCK', 'ROE', 'ROIV', 'ROKU', 'ROOT', 'ROST', 'RPD', 'RPRX', 'RRBI', 'RRR', 'RTAC', 'RTACU', 'RTH', 'RTXG', 'RUN', 'RUNN', 'RUSC', 'RUSHA', 'RUSHB', 'RVMD', 'RVNL', 'RWAY', 'RWAYL', 'RWAYZ', 'RYAAY', 'RYTM', 'SAFT', 'SAIC', 'SAIH', 'SAIL', 'SAMG', 'SANM', 'SARK', 'SATS', 'SBAC', 'SBCF', 'SBET', 'SBFG', 'SBGI', 'SBLK', 'SBRA', 'SBUX', 'SCDS', 'SCHL', 'SCLX', 'SCSC', 'SCVL', 'SCZ', 'SDG', 'SDGR', 'SDHI', 'SDHIU', 'SDM', 'SDSI', 'SDTY', 'SDVY', 'SEAT', 'SEDG', 'SEEM', 'SEIC', 'SEIE', 'SEIS', 'SENEA', 'SENEB', 'SEPN', 'SERV', 'SETM', 'SEVN', 'SEZL', 'SFBC', 'SFD', 'SFLO', 'SFM', 'SFNC', 'SFST', 'SGA', 'SGC', 'SGRY', 'SHBI', 'SHC', 'SHEN', 'SHOO', 'SHOP', 'SHPD', 'SHPU', 'SHRY', 'SHV', 'SHY', 'SIBN', 'SIFY', 'SIGI', 'SIGIP', 'SILC', 'SIMA', 'SIMAU', 'SIMO', 'SION', 'SIRI', 'SIXG', 'SJCP', 'SJLD', 'SKOR', 'SKWD', 'SKYT', 'SKYU', 'SKYW', 'SKYY', 'SLAB', 'SLDE', 'SLGL', 'SLM', 'SLMBP', 'SLNO', 'SLP', 'SLQD', 'SLRC', 'SLVO', 'SLVR', 'SMBC', 'SMCC', 'SMCF', 'SMCI', 'SMCL', 'SMCO', 'SMCX', 'SMHX', 'SMID', 'SMLR', 'SMMT', 'SMPL', 'SMRI', 'SMST', 'SMTC', 'SMTI', 'SNCY', 'SNDK', 'SNDX', 'SNEX', 'SNSR', 'SNWV', 'SNY', 'SOCAU', 'SOCL', 'SOFI', 'SOFX', 'SOGP', 'SOHOB', 'SOHON', 'SOHOO', 'SOHU', 'SOLT', 'SOLZ', 'SONO', 'SOUN', 'SOUX', 'SOXQ', 'SOXX', 'SPAM', 'SPAQ', 'SPBC', 'SPC', 'SPCB', 'SPCX', 'SPCY', 'SPEGU', 'SPFI', 'SPKL', 'SPKLU', 'SPNS', 'SPOK', 'SPRX', 'SPSC', 'SPT', 'SPTN', 'SPXD', 'SPYQ', 'SQFTP', 'SQLV', 'SQQQ', 'SRAD', 'SRBK', 'SRCE', 'SRDX', 'SRET', 'SRPT', 'SRRK', 'SRZN', 'SSBI', 'SSEAU', 'SSNC', 'SSRM', 'SSSSL', 'SSTI', 'SSYS', 'STAA', 'STBA', 'STEP', 'STLD', 'STNC', 'STNE', 'STOK', 'STRA', 'STRC', 'STRD', 'STRF', 'STRK', 'STRS', 'STRT', 'STRZ', 'STX', 'SUNS', 'SUPN', 'SUPP', 'SUPX', 'SUSB', 'SUSC', 'SUSL', 'SVCC', 'SVCCU', 'SVII', 'SVIIU', 'SWKH', 'SWKHL', 'SWKS', 'SWP', 'SYBT', 'SYM', 'SYNA', 'SYRE', 'SZZL', 'SZZLU', 'TACH', 'TACHU', 'TACO', 'TACOU', 'TARK', 'TARS', 'TASK', 'TATT', 'TAVI', 'TAVIU', 'TAX', 'TAXE', 'TAXI', 'TAXS', 'TAXT', 'TAYD', 'TBBK', 'TBCH', 'TBIL', 'TBLD', 'TBMC', 'TBPH', 'TBRG', 'TC', 'TCBI', 'TCBIO', 'TCBK', 'TCBS', 'TCBX', 'TCHI', 'TCMD', 'TCOM', 'TCX', 'TDAC', 'TDACU', 'TDI', 'TDIV', 'TDSB', 'TDSC', 'TDUP', 'TEAM', 'TECH', 'TECTP', 'TECX', 'TEKX', 'TEKY', 'TEM', 'TENB', 'TER', 'TEXN', 'TFNS', 'TFSL', 'TGTX', 'THFF', 'THMZ', 'THRM', 'THRY', 'TIGO', 'TIGR', 'TIL', 'TILE', 'TIPT', 'TITN', 'TLT', 'TMB', 'TMDX', 'TMED', 'TMET', 'TMUS', 'TMUSI', 'TMUSL', 'TMUSZ', 'TNDM', 'TNXP', 'TOWN', 'TPG', 'TPGXL', 'TPLS', 'TPST', 'TQQQ', 'TQQY', 'TREE', 'TRI', 'TRIN', 'TRINI', 'TRINZ', 'TRIP', 'TRMB', 'TRMD', 'TRMK']

tickers3 = ['TRML', 'TRNS', 'TROW', 'TRS', 'TRST', 'TRUD', 'TRUP', 'TRUT', 'TSAT', 'TSBK', 'TSCO', 'TSDD', 'TSEL', 'TSEM', 'TSL', 'TSLL', 'TSLR', 'TSMG', 'TSMU', 'TSMX', 'TSMZ', 'TSPY', 'TSSI', 'TTAN', 'TTD', 'TTEK', 'TTEQ', 'TTMI', 'TTWO', 'TUG', 'TUGN', 'TUR', 'TURF', 'TVA', 'TVACU', 'TVAI', 'TVAIU', 'TVRD', 'TVTX', 'TW', 'TWFG', 'TWIN', 'TWST', 'TXG', 'TXN', 'TXRH', 'TXSS', 'TXUE', 'TXUG', 'TYRA', 'TZOO', 'UAE', 'UAL', 'UBCP', 'UBND', 'UBRL', 'UBSI', 'UCRD', 'UCTT', 'UCYB', 'UEVM', 'UFCS', 'UFIV', 'UFO', 'UFPI', 'UFPT', 'UITB', 'UIVM', 'ULH', 'ULVM', 'UMBF', 'UMBFO', 'UMMA', 'UNB', 'UNHG', 'UNIY', 'UNTY', 'UPB', 'UPBD', 'UPGR', 'UPST', 'UPWK', 'URBN', 'URGN', 'URNJ', 'USAF', 'USAR', 'USAU', 'USCB', 'USCL', 'USDX', 'USFI', 'USIG', 'USIN', 'USLM', 'USMC', 'USOI', 'USRD', 'USSH', 'USTB', 'USVM', 'USVN', 'USXF', 'UTEN', 'UTHY', 'UTMD', 'UTRE', 'UTWO', 'UTWY', 'UVSP', 'UYLD', 'UYSC', 'UYSCU', 'VABK', 'VACH', 'VACHU', 'VALN', 'VALU', 'VBIL', 'VBNK', 'VBTX', 'VC', 'VCEL', 'VCIC', 'VCICU', 'VCIG', 'VCIT', 'VCLT', 'VCRB', 'VCSH', 'VCTR', 'VCYT', 'VECO', 'VEON', 'VERA', 'VERX', 'VFLO', 'VGIT', 'VGLT', 'VGSH', 'VGSR', 'VGUS', 'VIASP', 'VIAV', 'VICR', 'VIGI', 'VINP', 'VITL', 'VIXI', 'VKTX', 'VLGEA', 'VLY', 'VLYPN', 'VLYPO', 'VLYPP', 'VMBS', 'VNMEU', 'VNOM', 'VNQI', 'VOD', 'VOLT', 'VONG', 'VONV', 'VOR', 'VOTE', 'VPLS', 'VRDN', 'VREX', 'VRIG', 'VRM', 'VRNA', 'VRNS', 'VRNT', 'VRRM', 'VRSK', 'VRSN', 'VRTL', 'VSAT', 'VSDA', 'VSEC', 'VSMV', 'VSTL', 'VTC', 'VTHR', 'VTIP', 'VTVT', 'VTWG', 'VTWO', 'VTWV', 'VWAV', 'VWOB', 'VXUS', 'VYMI', 'WABC', 'WABF', 'WAFD', 'WAFDP', 'WASH', 'WAY', 'WB', 'WBD', 'WBTN', 'WCBR', 'WCLD', 'WDAY', 'WDC', 'WDFC', 'WEEI', 'WENN', 'WENNU', 'WERN', 'WEYS', 'WFCF', 'WFRD', 'WGMI', 'WGS', 'WHFCL', 'WHLRD', 'WHLRL', 'WILC', 'WING', 'WISE', 'WIX', 'WLAC', 'WLACU', 'WLDN', 'WLFC', 'WMG', 'WNEB', 'WOOD', 'WRD', 'WRLD', 'WRND', 'WSBC', 'WSBCP', 'WSBF', 'WSC', 'WSFS', 'WSML', 'WTBA', 'WTBN', 'WTFC', 'WTFCN', 'WTG', 'WTGUU', 'WTIP', 'WTMU', 'WTMY', 'WULF', 'WW', 'WWD', 'WYFI', 'WYNN', 'XAIX', 'XBIL', 'XBTY', 'XCNY', 'XEL', 'XENE', 'XFIX', 'XMAG', 'XMTR', 'XNCR', 'XOMA', 'XOMAO', 'XOMAP', 'XOMX', 'XOMZ', 'XOVR', 'XP', 'XPEL', 'XRAY', 'XRPI', 'XRPT', 'XT', 'XYZG', 'YB', 'YDES', 'YHNA', 'YHNAU', 'YLDE', 'YNOT', 'YOKE', 'YORW', 'YQQQ', 'YSPY', 'Z', 'ZAP', 'ZBIO', 'ZD', 'ZEUS', 'ZFZZT', 'ZG', 'ZIMV', 'ZION', 'ZIONP', 'ZIPP', 'ZJZZT', 'ZLAB', 'ZM', 'ZS', 'ZTEN', 'ZTOP', 'ZTRE', 'ZTWO', 'ZUMZ', 'ZVZZT', 'ZWZZT', 'ZXZZT', 'ZYME', 'ZZZ']


df_results = []

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):
        B = df.iloc[i]       # Breakout candle
        A = df.iloc[i-1]     # Candle A
        C = df.iloc[i+1]     # Candle C for FVG

        B_close = B['Close'].item()
        B_open = B['Open'].item()
        B_low = B['Low'].item()
        A_high = A['High'].item()
        A_open = A['Open'].item()
        A_low = A['Low'].item()
        C_open = C['Open'].item()
        C_low = C['Low'].item()
        breakout_date = df.index[i].date()

        if B_open == B_low or A_open == A_low or C_open == C_low or B_low > A_high:
            continue


        # 55-day high before breakout
        prev_55d_slice = df['High'].iloc[max(i-55,0):i]
        if prev_55d_slice.empty:
            continue
        prev_55d_high = prev_55d_slice.max()
        if isinstance(prev_55d_high, pd.Series):
            prev_55d_high = prev_55d_high.item()

        # Recent 10-day high before breakout
        recent_10d_slice = df['High'].iloc[max(i-10,0):i]
        recent_10d_high = recent_10d_slice.max()
        if isinstance(recent_10d_high, pd.Series):
            recent_10d_high = recent_10d_high.item()

        # ---------------------------
        # Breakout Conditions
        # ---------------------------
        if (
            B_close > prev_55d_high and
            B_close > B_open and
            recent_10d_high < prev_55d_high
        ):
            # ---------------------------
            # FVG Conditions
            # ---------------------------
            if C_low > A_high:  # no minimum % threshold
                fvg_low = A_high
                fvg_high = C_low

                # Look forward for FVG test starting AFTER Candle C
                test_date = None
                test_low = None
                for j in range(i+2, total_len):  # start from candle after C
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break

                df_results.append({
                    "Stock": ticker,
                    "Breakout Date": breakout_date,
                    "Prev 55D High": prev_55d_high,
                    "Recent 10-Day High": recent_10d_high,
                    "Breakout Close": B_close,
                    "Candle A (Date)": A.name.date(),
                    "Candle A High": A_high,
                    "Candle C (Date)": C.name.date(),
                    "Candle C Low": C_low,
                    "FVG Zone": f"{A_high} → {C_low}",
                    "FVG First Test Date": test_date,
                    "FVG First Test Low": test_low
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=False)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Print Latest FVG per stock
# ---------------------------
if df_results:
    df_results_df = pd.DataFrame(df_results)
    latest_fvg_df = df_results_df.sort_values("Breakout Date").groupby("Stock").tail(1)

    for idx, res in latest_fvg_df.iterrows():
        print("\n============================")
        print(f"📊 Latest FVG Zone: {res['Stock']}")
        print("----------------------------")
        for k, v in res.items():
            if k != "Stock":
                print(f"{k}: {v}")
else:
    print("⚠️ No valid bullish breakouts with meaningful FVG found in the last 10 days.")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


📊 Latest FVG Zone: ARQT
----------------------------
Breakout Date: 2025-07-10
Prev 55D High: 14.6899995803833
Recent 10-Day High: 14.64799976348877
Breakout Close: 15.270000457763672
Candle A (Date): 2025-07-09
Candle A High: 14.529999732971191
Candle C (Date): 2025-07-11
Candle C Low: 14.699999809265137
FVG Zone: 14.529999732971191 → 14.699999809265137
FVG First Test Date: None
FVG First Test Low: nan

📊 Latest FVG Zone: ASTS
----------------------------
Breakout Date: 2025-07-17
Prev 55D High: 54.04999923706055
Recent 10-Day High: 52.6879997253418
Breakout Close: 57.45000076293945
Candle A (Date): 2025-07-16
Candle A High: 52.6879997253418
Candle C (Date): 2025-07-18
Candle C Low: 55.209999084472656
FVG Zone: 52.6879997253418 → 55.209999084472656
FVG First Test Date: 2025-07-25
FVG First Test Low: 53.029998779296875

📊 Latest FVG Zone: AVTX
----------------------------
Breakout Date: 2025-07-17
Prev 55D High: 5.050000190734863
Recent 10-Day High: 5.019999980926514
Breakout Close: 5

In [ ]:
# Filter results where FVG First Test Date is NOT None
fvg_tested_df = df_results_df[df_results_df["FVG First Test Date"].notna()].copy()

if not fvg_tested_df.empty:
    print("\n📊 Stocks with FVG Zone TESTED:\n")
    for idx, res in fvg_tested_df.iterrows():
        print("============================")
        print(f"Stock: {res['Stock']}")
        print("----------------------------")
        for k, v in res.items():
            if k != "Stock":
                print(f"{k}: {v}")
else:
    print("⚠️ No FVG zones have been tested yet.")


📊 Stocks with FVG Zone TESTED:

Stock: ACLX
----------------------------
Breakout Date: 2025-09-11
Prev 55D High: 74.87999725341797
Recent 10-Day High: 73.0999984741211
Breakout Close: 77.68000030517578
Candle A (Date): 2025-09-10
Candle A High: 72.80999755859375
Candle C (Date): 2025-09-12
Candle C Low: 76.05999755859375
FVG Zone: 72.80999755859375 → 76.05999755859375
FVG First Test Date: 2025-09-15
FVG First Test Low: 73.16000366210938
Stock: ADUR
----------------------------
Breakout Date: 2025-09-09
Prev 55D High: 12.399999618530273
Recent 10-Day High: 12.050000190734863
Breakout Close: 12.949999809265137
Candle A (Date): 2025-09-08
Candle A High: 11.800000190734863
Candle C (Date): 2025-09-10
Candle C Low: 12.640000343322754
FVG Zone: 11.800000190734863 → 12.640000343322754
FVG First Test Date: 2025-09-11
FVG First Test Low: 12.5600004196167
Stock: ALTY
----------------------------
Breakout Date: 2025-08-22
Prev 55D High: 11.789999961853027
Recent 10-Day High: 11.77400016784668
B

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 10  # recent 10-day high
today = datetime.today().date()
start_date = today - timedelta(days=55 + lookback_days + 30)  # buffer for 55D + 10D



tickers = ['AADR', 'AAON', 'AAUS', 'AAXJ', 'ABNB', 'ABVX', 'ACGL', 'ACIW', 'ACLS', 'ACLX', 'ACWI', 'ACWX', 'ADUS', 'AEIS', 'AEP', 'AFRM', 'AGYS', 'AIA', 'AIRR', 'AKAM', 'ALGN', 'ALGT', 'ALRM', 'AMAT', 'AMBA', 'AMD', 'AMSC', 'AMWD', 'ANIP', 'AOTG', 'APPX', 'ARCB', 'ARM', 'ARVR', 'ASND', 'ATLC', 'AUMI', 'AVGB', 'AVGX', 'AVL', 'AVT', 'AVUQ', 'AVXC', 'AXSM', 'AZN', 'BANF', 'BANR', 'BBH', 'BBIO', 'BCLO', 'BCPC', 'BELFA', 'BELFB', 'BFC', 'BHF', 'BHRB', 'BIB', 'BIDU', 'BIIB', 'BITS', 'BKCH', 'BL', 'BLBD', 'BLKB', 'BLTE', 'BMRN', 'BND', 'BNDW', 'BNTX', 'BOKF', 'BPOP', 'BRHY', 'BRTR', 'BSY', 'BTFX', 'BTSGU', 'CAKE', 'CALI', 'CALM', 'CAMT', 'CAR', 'CARZ', 'CASH', 'CATH', 'CBSH', 'CCB', 'CCEP', 'CDC', 'CDL', 'CDTX', 'CDW', 'CELC', 'CELH', 'CFA', 'CFFI', 'CFO', 'CG', 'CHCO', 'CHDN', 'CHEF', 'CHKP', 'CHMG', 'CHRD', 'CHRW', 'CIBR', 'CIGI', 'CIL', 'CINF', 'CLMB', 'CLOA', 'CMPR', 'CNXC', 'CNXN', 'COKE', 'COLM', 'COO', 'CORT', 'CPAG', 'CPHY', 'CRDO', 'CROX', 'CRSP', 'CRUS', 'CRVL', 'CRWV', 'CSB', 'CSCO', 'CSGP', 'CSGS', 'CTBI', 'CTSH', 'CVLT', 'CWST', 'CZFS', 'DDOG', 'DFGP', 'DFGX', 'DGCB', 'DGRS', 'DGRW', 'DHIL', 'DIOD', 'DLTR', 'DMXF', 'DOCU', 'DOOO', 'DORM', 'DOX', 'DRUG', 'DSGX', 'DVY', 'DWAS', 'DWUS', 'DXCM', 'DXPE', 'EA', 'EBAY', 'EBI', 'EEFT', 'EEMA', 'EFSC', 'EGGQ', 'EMB', 'EMCB', 'EMXC', 'ENSG', 'ENTG', 'EQRR', 'ESEA', 'ESGD', 'ESGU', 'ESPO', 'ESQ', 'ETSY', 'EVRG', 'EVSD', 'EWBC', 'EXAS', 'EXE', 'EXEEL', 'EXEEW', 'EXEEZ', 'EXPO', 'FAB', 'FAD', 'FANG', 'FBIZ', 'FBNC', 'FCFS', 'FDCF', 'FDT', 'FDTS', 'FELE', 'FEP', 'FER', 'FEUZ', 'FEX', 'FFUT', 'FGM', 'FIVE', 'FJP', 'FLDB', 'FLEX', 'FMB', 'FMUB', 'FNK', 'FNX', 'FNY', 'FORTY', 'FOX', 'FOXA', 'FPXI', 'FRHC', 'FRPT', 'FSZ', 'FTA', 'FTAI', 'FTC', 'FTCS', 'FTDR', 'FTDS', 'FTNT', 'FTSM', 'FTXL', 'FUTU', 'FV', 'FWONA', 'FWONK', 'FYC', 'FYT', 'FYX', 'GEHC', 'GGLL', 'GH', 'GILD', 'GLDI', 'GPIQ', 'GPIX', 'GPRF', 'GRID', 'GRVY', 'GSBC', 'GSHD', 'GTLB', 'HALO', 'HAS', 'HBCP', 'HEQQ', 'HLAL', 'HLNE', 'HOLX', 'HOOD', 'HOOG', 'HOOX', 'HQGO', 'HQY', 'HSIC', 'HURN', 'HWC', 'HWKN', 'HYBI', 'IBB', 'IBIT', 'IBKR', 'IBOC', 'ICFI', 'ICLR', 'ICUI', 'IEF', 'IEI', 'IEUS', 'IGF', 'IGIB', 'IGSB', 'IJT', 'ILMN', 'IMCV', 'IMKTA', 'INCY', 'INDB', 'INDY', 'INOD', 'INSM', 'IONL', 'IONS', 'IONX', 'IOSP', 'IPAR', 'IPGP', 'IPKW', 'IRMD', 'IRON', 'IRTC', 'ISHG', 'ITRI', 'IUS', 'IUSG', 'IUSV', 'IXUS', 'JAZZ', 'JBHT', 'JBSS', 'JDOC', 'JEPQ', 'JGLO', 'JIVE', 'JJSF', 'JKHY', 'JOYY', 'JPEF', 'JSMD', 'JSML', 'JTEK', 'KALU', 'KARO', 'KAT', 'KBWB', 'KBWP', 'KBWR', 'KMB', 'KRUS', 'KRYS', 'KSPI', 'KTOS', 'LAMR', 'LBRDA', 'LBRDK', 'LCDS', 'LDEM', 'LEGR', 'LGIH', 'LGND', 'LIF', 'LITE', 'LIVN', 'LKFN', 'LLYVA', 'LLYVK', 'LMAT', 'LMB', 'LNT', 'LNTH', 'LNW', 'LOGI', 'LRCX', 'LRGE', 'LSCC', 'LSTR', 'LULU', 'MASI', 'MBB', 'MCDS', 'MCHI', 'MCHP', 'MCHPP', 'MCRI', 'MDLZ', 'MENS', 'MGEE', 'MGRC', 'MIDD', 'MILN', 'MIRM', 'MKSI', 'MKTX', 'MLAB', 'MLTX', 'MMSI', 'MMYT', 'MNPR', 'MNST', 'MQQQ', 'MRCY', 'MRUS', 'MRVL', 'MSEX', 'MSFU', 'MTSI', 'MU', 'MYRG', 'MZTI', 'NATH', 'NBIS', 'NBIX', 'NBN', 'NDAQ', 'NEGG', 'NFTY', 'NFXL', 'NICE', 'NKTR', 'NOVT', 'NRIM', 'NSIT', 'NTAP', 'NTES', 'NTNX', 'NTRA', 'NTRS', 'NUTX', 'NUVL', 'NVDA', 'NVDL', 'NVDU', 'NVEC', 'NWE', 'NWPX', 'NXT', 'NXTG', 'OBIL', 'ODD', 'ODFL', 'OKLL', 'OKTA', 'OLED', 'OLLI', 'OMAB', 'ON', 'ONEQ', 'ORLY', 'OTTR', 'OZK', 'PABD', 'PABU', 'PATK', 'PAYX', 'PCAR', 'PCMM', 'PCTY', 'PDD', 'PDP', 'PEGA', 'PEP', 'PEZ', 'PFBC', 'PFG', 'PFI', 'PFIS', 'PFM', 'PHO', 'PI', 'PKW', 'PLMR', 'PLTR', 'PLTU', 'PLUS', 'PLXS', 'PNFP', 'PNQI', 'PNRG', 'PPH', 'PRN', 'PSC', 'PSCD', 'PSCF', 'PSCI', 'PSCM', 'PSCT', 'PSCU', 'PSET', 'PSIX', 'PSL', 'PSMT', 'PTCT', 'PTF', 'PTGX', 'PTNQ', 'PVLA', 'PWRD', 'PY', 'PYPL', 'PYZ', 'QABA', 'QCOM', 'QCRH', 'QLYS', 'QMMM', 'QMOM', 'QQA', 'QQEW', 'QQHG', 'QQQE', 'QQQH', 'QQQI', 'QQQP', 'QQUP', 'QQXT', 'QRVO', 'QTUM', 'RBCAA', 'RBIL', 'RDNT', 'RDTL', 'RDVT', 'RDVY', 'REG', 'RFDI', 'RFEM', 'RFEU', 'RGEN', 'RGLD', 'RGTX', 'RHLD', 'RING', 'RKLX', 'RMBS', 'RNEM', 'ROAD', 'ROBT', 'ROCK', 'ROKU', 'ROOT', 'ROST', 'RRBI', 'RRR', 'RUSHA', 'RUSHB', 'RYAAY', 'RYTM', 'SAFT', 'SAIC', 'SANM', 'SATS', 'SBAC', 'SBUX', 'SCDS', 'SCZ', 'SDG', 'SDSI', 'SEIC', 'SENEA', 'SENEB', 'SEZL', 'SFM', 'SHOP', 'SHV', 'SHY', 'SIGI', 'SIMO', 'SIXG', 'SKYW', 'SKYY', 'SLAB', 'SLMBP', 'SLNO', 'SLQD', 'SLVO', 'SMBC', 'SMTC', 'SNDK', 'SNEX', 'SOCL', 'SPAQ', 'SPSC', 'SPYQ', 'SRCE', 'SSNC', 'STEP', 'STLD', 'STRA', 'STRC', 'STRD', 'STRF', 'STRK', 'STRT', 'SUPP', 'SUPX', 'SUSL', 'SWKS', 'SYBT', 'SYM', 'SYNA', 'TARK', 'TARS', 'TAXE', 'TAXI', 'TAXS', 'TAXT', 'TBBK', 'TCBI', 'TCOM', 'TDIV', 'TEAM', 'TECH', 'TEM', 'TER', 'THFF', 'TLT', 'TMDX', 'TPG', 'TQQQ', 'TREE', 'TRI', 'TRMB', 'TRNS', 'TROW', 'TSCO', 'TSEM', 'TTAN', 'TTMI', 'TW', 'TXN', 'TXRH', 'UAL', 'UCYB', 'UEVM', 'UFPI', 'UFPT', 'UIVM', 'ULVM', 'UMBF', 'UNTY', 'UPST', 'URBN', 'USCL', 'USIG', 'USIN', 'USLM', 'USMC', 'USOI', 'USSH', 'USTB', 'USVM', 'USXF', 'UTMD', 'UYLD', 'VBIL', 'VC', 'VCIT', 'VCLT', 'VCRB', 'VCSH', 'VCTR', 'VEON', 'VGIT', 'VGLT', 'VGSH', 'VGUS', 'VICR', 'VIGI', 'VONG', 'VONV', 'VOTE', 'VPLS', 'VRNA', 'VRNS', 'VSDA', 'VSEC', 'VSMV', 'VTC', 'VTIP', 'VTWO', 'VTWV', 'VWOB', 'VXUS', 'VYMI', 'WDC', 'WFRD', 'WGS', 'WHLRL', 'WIX', 'WLDN', 'WLFC', 'WOOD', 'WRLD', 'WSFS', 'WTFC', 'WYNN', 'XBIL', 'XEL', 'XFIX', 'XMTR', 'XT', 'YLDE', 'Z', 'ZG', 'ZION', 'ZM', 'ZTEN', 'ZTOP', 'ZTRE', 'ZTWO']

df_results = []

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):
        B = df.iloc[i]       # Breakout candle
        A = df.iloc[i-1]     # Candle A
        C = df.iloc[i+1]     # Candle C for FVG

        B_close = B['Close'].item()
        B_open = B['Open'].item()
        B_low = B['Low'].item()
        A_high = A['High'].item()
        A_open = A['Open'].item()
        A_low = A['Low'].item()
        C_open = C['Open'].item()
        C_low = C['Low'].item()
        breakout_date = df.index[i].date()

        if B_open == B_low or A_open == A_low or C_open == C_low or B_low > A_high or B_close < (B_open * 1.01):
            continue


        # 55-day high before breakout
        prev_55d_slice = df['High'].iloc[max(i-55,0):i]
        if prev_55d_slice.empty:
            continue
        prev_55d_high = prev_55d_slice.max()
        if isinstance(prev_55d_high, pd.Series):
            prev_55d_high = prev_55d_high.item()

        # Recent 10-day high before breakout
        recent_10d_slice = df['High'].iloc[max(i-10,0):i]
        recent_10d_high = recent_10d_slice.max()
        if isinstance(recent_10d_high, pd.Series):
            recent_10d_high = recent_10d_high.item()

        # ---------------------------
        # Breakout Conditions
        # ---------------------------
        if (
            B_close > prev_55d_high and
            B_close > B_open and
            recent_10d_high < prev_55d_high
        ):
            # ---------------------------
            # FVG Conditions
            # ---------------------------
            if C_low > A_high:  # no minimum % threshold
                fvg_low = A_high
                fvg_high = C_low

                # Look forward for FVG test starting AFTER Candle C
                test_date = None
                test_low = None
                for j in range(i+2, total_len):  # start from candle after C
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break

                df_results.append({
                    "Stock": ticker,
                    "Breakout Date": breakout_date,
                    "Prev 55D High": prev_55d_high,
                    "Recent 10-Day High": recent_10d_high,
                    "Breakout Close": B_close,
                    "Candle A (Date)": A.name.date(),
                    "Candle A High": A_high,
                    "Candle C (Date)": C.name.date(),
                    "Candle C Low": C_low,
                    "FVG Zone": f"{A_high} → {C_low}",
                    "FVG First Test Date": test_date,
                    "FVG First Test Low": test_low
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=False)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Print Latest FVG per stock where FVG has been tested
# ---------------------------
if df_results:
    df_results_df = pd.DataFrame(df_results)
    fvg_tested_df = df_results_df[df_results_df["FVG First Test Date"].notna()].copy()

    if not fvg_tested_df.empty:
        print("\n📊 Stocks with FVG Zone TESTED:\n")
        for idx, res in fvg_tested_df.iterrows():
            print("============================")
            print(f"Stock: {res['Stock']}")
            print("----------------------------")
            for k, v in res.items():
                if k != "Stock":
                    print(f"{k}: {v}")
    else:
        print("⚠️ No FVG zones have been tested yet.")
else:
    print("⚠️ No valid bullish breakouts with meaningful FVG found in the last 10 days.")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


📊 Stocks with FVG Zone TESTED:

Stock: ACLX
----------------------------
Breakout Date: 2025-09-11
Prev 55D High: 74.87999725341797
Recent 10-Day High: 73.0999984741211
Breakout Close: 77.68000030517578
Candle A (Date): 2025-09-10
Candle A High: 72.80999755859375
Candle C (Date): 2025-09-12
Candle C Low: 76.05999755859375
FVG Zone: 72.80999755859375 → 76.05999755859375
FVG First Test Date: 2025-09-15
FVG First Test Low: 73.16000366210938
Stock: AXSM
----------------------------
Breakout Date: 2025-08-20
Prev 55D High: 114.66400146484375
Recent 10-Day High: 112.47000122070312
Breakout Close: 116.98999786376953
Candle A (Date): 2025-08-19
Candle A High: 112.08999633789062
Candle C (Date): 2025-08-21
Candle C Low: 114.48999786376953
FVG Zone: 112.08999633789062 → 114.48999786376953
FVG First Test Date: 2025-09-16
FVG First Test Low: 112.9000015258789
Stock: CG
----------------------------
Breakout Date: 2025-09-11
Prev 55D High: 65.97000122070312
Recent 10-Day High: 65.86000061035156
Bre

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30  # recent 30-day high
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days + 30)  # buffer for 52w + 30d

tickers = ['AACB', 'AACBU', 'AACI', 'AACIU', 'AADR', 'AAL', 'AALG', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPG', 'AAPL', 'AAPU', 'AARD', 'AAUS', 'AAVM', 'AAXJ', 'ABCS', 'ABI', 'ABIG', 'ABLLL', 'ABNB', 'ABVX', 'ACAD', 'ACFN', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACIC', 'ACIW', 'ACLS', 'ACLX', 'ACMR', 'ACNB', 'ZIPP', 'ZJZZT', 'ZLAB', 'ZM', 'ZS', 'ZTEN', 'ZTOP', 'ZTRE', 'ZTWO', 'ZUMZ', 'ZVZZT', 'ZWZZT', 'ZXZZT', 'ZYME', 'ZZZ']

df_results = []

# ---------------------------
# Function to check FVG + FVG test
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):
        B = df.iloc[i]       # Breakout candle
        A = df.iloc[i-1]     # Candle A
        C = df.iloc[i+1]     # Candle C for FVG

        B_close = B['Close'].item()
        B_open = B['Open'].item()
        B_low = B['Low'].item()
        A_high = A['High'].item()
        A_open = A['Open'].item()
        A_low = A['Low'].item()
        C_open = C['Open'].item()
        C_low = C['Low'].item()
        breakout_date = df.index[i].date()

        if B_open == B_low or A_open == A_low or C_open == C_low or B_low > A_high or B_close < (B_open * 1.02) or C_low - A_high < 4:
            continue

        # 52-week high before breakout (approx 252 trading days)
        prev_52w_slice = df['High'].iloc[max(i-252,0):i]
        if prev_52w_slice.empty:
            continue
        prev_52w_high = prev_52w_slice.max()
        if isinstance(prev_52w_high, pd.Series):
            prev_52w_high = prev_52w_high.item()

        # Recent 30-day high before breakout
        recent_30d_slice = df['High'].iloc[max(i-30,0):i]
        recent_30d_high = recent_30d_slice.max()
        if isinstance(recent_30d_high, pd.Series):
            recent_30d_high = recent_30d_high.item()

        # ---------------------------
        # Breakout Conditions
        # ---------------------------
        if (
            B_close > prev_52w_high and
            B_close > B_open and
            recent_30d_high < prev_52w_high
        ):
            # ---------------------------
            # FVG Conditions
            # ---------------------------
            if C_low > A_high:  # no minimum % threshold
                fvg_low = A_high
                fvg_high = C_low

                # Look forward for FVG test starting AFTER Candle C
                test_date = None
                test_low = None
                for j in range(i+2, total_len):  # start from candle after C
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break

                df_results.append({
                    "Stock": ticker,
                    "Breakout Date": breakout_date,
                    "Prev 52W High": prev_52w_high,
                    "Recent 30-Day High": recent_30d_high,
                    "Breakout Close": B_close,
                    "Candle A (Date)": A.name.date(),
                    "Candle A High": A_high,
                    "Candle C (Date)": C.name.date(),
                    "Candle C Low": C_low,
                    "FVG Zone": f"{A_high} → {C_low}",
                    "FVG First Test Date": test_date,
                    "FVG First Test Low": test_low
                })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=False)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Print Latest FVG per stock where FVG has been tested
# ---------------------------
if df_results:
    df_results_df = pd.DataFrame(df_results)
    fvg_tested_df = df_results_df[df_results_df["FVG First Test Date"].notna()].copy()

    if not fvg_tested_df.empty:
        print("\n📊 Stocks with FVG Zone TESTED:\n")
        for idx, res in fvg_tested_df.iterrows():
            print("============================")
            print(f"Stock: {res['Stock']}")
            print("----------------------------")
            for k, v in res.items():
                if k != "Stock":
                    print(f"{k}: {v}")
    else:
        print("⚠️ No FVG zones have been tested yet.")
else:
    print("⚠️ No valid bullish breakouts with meaningful FVG found in the last 30 days.")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


📊 Stocks with FVG Zone TESTED:

Stock: ACFN
----------------------------
Breakout Date: 2025-07-28
Prev 52W High: 19.5
Recent 30-Day High: 18.549999237060547
Breakout Close: 21.950000762939453
Candle A (Date): 2025-07-25
Candle A High: 18.010000228881836
Candle C (Date): 2025-07-29
Candle C Low: 23.18000030517578
FVG Zone: 18.010000228881836 → 23.18000030517578
FVG First Test Date: 2025-08-01
FVG First Test Low: 22.59000015258789


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# ---------------------------
# Parameters
# ---------------------------
lookback_days = 30  # recent 30-day high
today = datetime.today().date()
start_date = today - timedelta(days=365 + lookback_days + 30)  # buffer for 52w + 30d

# Reduced list of tickers for demonstration and stability
# tickers = ["SOFI", "JOBY", "FORM", "AAPL", "MSFT"]
tickers = ['AACB', 'AACBU', 'AACI', 'AACIU', 'AADR', 'AAL', 'AALG', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPG', 'AAPL', 'AAPU', 'AARD', 'AAUS', 'AAVM', 'AAXJ', 'ABCS', 'ABI', 'ABIG', 'ABLLL', 'ABNB', 'ABVX', 'ACAD', 'ACFN', 'ACGL', 'ACGLN', 'ACGLO', 'ACHC', 'ACIC', 'ACIW', 'ACLS', 'ACLX', 'ACMR', 'ACNB', 'ZIPP', 'ZJZZT', 'ZLAB', 'ZM', 'ZS', 'ZTEN', 'ZTOP', 'ZTRE', 'ZTWO', 'ZUMZ', 'ZVZZT', 'ZWZZT', 'ZXZZT', 'ZYME', 'ZZZ']

df_results = []

# ---------------------------
# Function to check FVG + FVG test + New Condition
# ---------------------------
def check_fvg_at_breakout(df, ticker):
    df = df.copy()
    total_len = len(df)

    for i in range(1, total_len-1):
        B = df.iloc[i]       # Breakout candle
        A = df.iloc[i-1]     # Candle A
        C = df.iloc[i+1]     # Candle C for FVG

        B_close = B['Close'].item()
        B_open = B['Open'].item()
        B_low = B['Low'].item()
        B_high = B['High'].item() # Get breakout day high
        A_high = A['High'].item()
        A_open = A['Open'].item()
        A_low = A['Low'].item()
        C_open = C['Open'].item()
        C_low = C['Low'].item()
        breakout_date = df.index[i].date()

        if B_open == B_low or A_open == A_low or C_open == C_low or B_low > A_high:
            continue

        # 52-week high before breakout (approx 252 trading days)
        prev_52w_slice = df['High'].iloc[max(i-252,0):i]
        if prev_52w_slice.empty:
            continue
        prev_52w_high = prev_52w_slice.max()
        if isinstance(prev_52w_high, pd.Series):
            prev_52w_high = prev_52w_high.item()


        # Recent 30-day high before breakout
        recent_30d_slice = df['High'].iloc[max(i-30,0):i]
        recent_30d_high = recent_30d_slice.max()
        if isinstance(recent_30d_high, pd.Series):
            recent_30d_high = recent_30d_high.item()

        # ---------------------------
        # Breakout Conditions
        # ---------------------------
        if (
            B_close > prev_52w_high and
            B_close > B_open and
            recent_30d_high < prev_52w_high
        ):
            # ---------------------------
            # FVG Conditions
            # ---------------------------
            if C_low > A_high:  # no minimum % threshold
                fvg_low = A_high
                fvg_high = C_low

                # Look forward for FVG test starting AFTER Candle C
                test_date = None
                test_low = None
                for j in range(i+2, total_len):  # start from candle after C
                    candle = df.iloc[j]
                    candle_low = candle['Low'].item()
                    if fvg_low <= candle_low <= fvg_high:
                        test_date = df.index[j].date()
                        test_low = candle_low
                        break

                # ---------------------------
                # New Condition: Max High after Breakout
                # ---------------------------
                max_high_after_breakout = None
                if test_date:
                    # Find the index of the test_date in the dataframe
                    test_date_index = df.index.get_loc(pd.to_datetime(test_date))
                    # Get the slice of Highs between breakout date and test date (inclusive)
                    high_slice_after_breakout = df['High'].iloc[i : test_date_index + 1]
                    if not high_slice_after_breakout.empty:
                        max_high_after_breakout = high_slice_after_breakout.max()
                        if isinstance(max_high_after_breakout, pd.Series):
                             max_high_after_breakout = max_high_after_breakout.item()


                # Check the new condition
                if max_high_after_breakout is not None and max_high_after_breakout > B_high * 1.1:

                    df_results.append({
                        "Stock": ticker,
                        "Breakout Date": breakout_date,
                        "Prev 52W High": prev_52w_high,
                        "Recent 30-Day High": recent_30d_high,
                        "Breakout Close": B_close,
                        "Breakout High": B_high, # Add breakout high to results
                        "Candle A (Date)": A.name.date(),
                        "Candle A High": A_high,
                        "Candle C (Date)": C.name.date(),
                        "Candle C Low": C_low,
                        "FVG Zone": f"{A_high} → {C_low}",
                        "FVG First Test Date": test_date,
                        "FVG First Test Low": test_low,
                        "Max High After Breakout": max_high_after_breakout # Add max high after breakout
                    })

# ---------------------------
# Main Loop
# ---------------------------
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=today, auto_adjust=False)
    if not data.empty:
        check_fvg_at_breakout(data, ticker)

# ---------------------------
# Print Latest FVG per stock where FVG has been tested
# ---------------------------
if df_results:
    df_results_df = pd.DataFrame(df_results)
    # Filter for results where the new condition was met (implicitly done by appending to df_results)

    if not df_results_df.empty:
        print("\n📊 Stocks meeting all conditions (52W Bullish Breakout + FVG + FVG Tested + High > Breakout High * 1.1):\n")
        for idx, res in df_results_df.iterrows():
            print("============================")
            print(f"Stock: {res['Stock']}")
            print("----------------------------")
            for k, v in res.items():
                if k != "Stock":
                    print(f"{k}: {v}")
    else:
        print("⚠️ No stocks found meeting all conditions.")
else:
    print("⚠️ No valid bullish breakouts with meaningful FVG found in the last 30 days.")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********


📊 Stocks meeting all conditions (52W Bullish Breakout + FVG + FVG Tested + High > Breakout High * 1.1):

Stock: ACFN
----------------------------
Breakout Date: 2025-07-28
Prev 52W High: 19.5
Recent 30-Day High: 18.549999237060547
Breakout Close: 21.950000762939453
Breakout High: 21.950000762939453
Candle A (Date): 2025-07-25
Candle A High: 18.010000228881836
Candle C (Date): 2025-07-29
Candle C Low: 23.18000030517578
FVG Zone: 18.010000228881836 → 23.18000030517578
FVG First Test Date: 2025-08-01
FVG First Test Low: 22.59000015258789
Max High After Breakout: 28.729999542236328
